In [22]:
#MASTER VARIABLES AND FUNCTIONS

#Imports
import ee
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
ee.Authenticate()
import geemap
import os
import time

#Optional for if executing in colab cells
from GEE_ImageFusion_Fxns_81322.core_functions_81322 import *
from GEE_ImageFusion_Fxns_81322.get_paired_collections_362023 import *
from GEE_ImageFusion_Fxns_81322.prep_functions_81322 import *


#Global Variables

# define training data temporal bounds broadly

startDate_ls7 = ee.String('1999-05-28')
endDate_ls7 = ee.String('2003-05-31'); # End date of ls7 full images
endDate_ls7_S2 = ee.String('2017-03-28'); # End date of ls7 full images


#   common bands between sensors that will be used for fusion

threshold = ee.Number(10)
thresholdS = ee.String('10')

road = ee.FeatureCollection("TIGER/2016/Roads")


#  landsat band names including qc band for masking
bandNamesLandsat = ee.List(['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'QA_PIXEL'])

landsatBands57 = ee.List(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL'])
landsatBands89 = ee.List(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL'])

# Sentinel Band Names
S2Bands = ['B2','B3','B4','B8','B11','B12', 'QA60']
bandNamesS2 = ['blue','green','red','nir','swir1','swir2', 'QA60']

#  modis band names
# bandNamesModis = ee.List(['blue', 'green', 'red', 'nir', 'swir1', 'swir2'])
# modisBands = ee.List([2, 3, 0, 1, 5, 6])
bandNamesModisT = ee.List(['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'QA_PIXEL'])
modisBandsT = ee.List(['sur_refl_b03', 'sur_refl_b04', 'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b06', 'sur_refl_b07', 'state_1km'])
bandNamesModisA = ee.List(['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'QA_PIXEL'])
modisBandsA = ee.List(['sur_refl_b03', 'sur_refl_b04', 'sur_refl_b01', 'sur_refl_b02', 'sur_refl_b06', 'sur_refl_b07', 'StateQA'])

commonBandNames = ee.List(['blue', 'green', 'red', 'nir', 'swir1', 'swir2']) #'r_gb', 

#For prefiltering global collections
conusSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/conusFusionSites_filtered')

modisCollection = ee.ImageCollection('MODIS/061/MOD09GA')\
    .select(modisBandsT, bandNamesModisT)\
    .merge(ee.ImageCollection("MODIS/061/MYD09A1").select(modisBandsA, bandNamesModisA))\
    .filterBounds(conusSites.geometry())

ls9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")\
        .filterBounds(conusSites.geometry())\
        .select(landsatBands89, bandNamesLandsat)

ls8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
        .filterBounds(conusSites.geometry())\
        .select(landsatBands89, bandNamesLandsat)

ls7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')\
    .filterBounds(conusSites.geometry())\
    .filterDate(startDate_ls7, endDate_ls7_S2)\
    .select(landsatBands57, bandNamesLandsat)
    
ls5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')\
        .filterBounds(conusSites.geometry())\
        .select(landsatBands57, bandNamesLandsat)

ls89 = ee.ImageCollection(ls9.merge(ls8).merge(ls7))
ls57 = ee.ImageCollection(ls7.merge(ls5))

s2_sr_col = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")


CLOUD_FILTER = 80
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 75

# radius of moving window
# Note: Generally, larger windows are better but as the window size increases,
# so does the memory requirement and we quickly will surpass the memory
# capacity of a single node (in testing 13 was max size for single band, and
# 10 was max size for up to 6 bands)
kernelRadius = ee.Number(8)
kernel = ee.Kernel.square(kernelRadius)
numPixels = kernelRadius.add(kernelRadius.add(1)).pow(2)

# number of land cover classes in scene
coverClasses = 6


datesStag = (('2013-03-18', '2013-11-01'),
        ('2013-10-01', '2014-05-01'),
        ('2014-04-01', '2014-11-01'),
        ('2014-10-01', '2015-05-01'),
        ('2015-04-01', '2015-11-01'),
        ('2015-10-01', '2016-05-01'),
        ('2016-04-01', '2016-11-01'),
        ('2016-10-01', '2017-05-01'),
        ('2017-04-01', '2017-11-01'),
        ('2017-10-01', '2018-05-01'),
        ('2018-04-01', '2018-11-01'),
        ('2018-10-01', '2019-05-01'),
        ('2019-04-01', '2019-11-01'),
        ('2019-10-01', '2020-05-01'),
        ('2020-04-01', '2020-11-01'),
        ('2020-10-01', '2021-05-01'),
        ('2021-04-01', '2021-11-01'),
        ('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))

# datesStag = (('2000-01-01', '2000-11-01'),
#         ('2000-10-01', '2001-05-01'),
#         ('2001-04-01', '2001-11-01'),
#         ('2001-10-01', '2002-05-01'),
#         ('2002-04-01', '2002-11-01'),
#         ('2002-10-01', '2003-05-01'),
#         ('2003-04-01', '2003-11-01'),
#         ('2003-10-01', '2004-05-01'),
#         ('2004-04-01', '2004-11-01'),
#         ('2004-10-01', '2005-05-01'),
#         ('2005-04-01', '2005-11-01'),
#         ('2005-10-01', '2006-05-01'),
#         ('2006-04-01', '2006-11-01'),
#         ('2006-10-01', '2007-05-01'),
#         ('2007-04-01', '2007-11-01'),
#         ('2007-10-01', '2008-05-01'),
#         ('2008-04-01', '2008-11-01'),
#         ('2008-10-01', '2009-05-01'),
#         ('2009-04-01', '2009-11-01'),
#         ('2009-10-01', '2010-05-01'),
#         ('2010-04-01', '2010-11-01'),
#         ('2010-10-01', '2011-05-01'),
#         ('2011-04-01', '2011-11-01'),
#         ('2011-10-01', '2012-05-01'),
#         ('2012-04-01', '2012-11-01'),
#         ('2012-10-01', '2013-03-18'))



#Master Script Export Inside Functions 
def sites_processing(startDate, endDate):
    def sortCols(site):
                
        region = ee.Feature(site).geometry()

        sortedImages = getPaired(startDate, endDate, ls89, ls57, 
              s2_sr_col, CLOUD_FILTER,
              landsatBands57, landsatBands89, bandNamesLandsat,
              S2Bands, bandNamesS2,
              modisCollection,
              commonBandNames,
              region, threshold) #s2_cloudless_col,

        #Create list of list of lists of groups of images for prediction
        subs = makeSubcollections(sortedImages)

        #Remove lists with no prediction images
        subs = subs.map(lambda el: ee.List(el).map(lambda subEl: ee.List(subEl).remove('null')).remove([]))

        #Turn into feature collection with size property to differentiate from complete and null image groups
        subsFeat = ee.FeatureCollection(ee.List(subs).map(lambda el: ee.Feature(None, {'subs': ee.List(el), 'subSize': ee.List(el).size()})))

        #Filter and remove null groups and extra properties
        subsFeat = subsFeat.filter(ee.Filter.gt('subSize', 2)).map(lambda feat: removeProperty(feat, 'subSize'))
                
        #return subsFeat
        return prediction(subsFeat, site) #, subList
    
    return sortCols


def prediction(subsFeat, site):
    
    
    def subPred(el): # list of lists of group of images to predict
        

        el = ee.List(el.toDictionary().values())
        sdist = ee.Feature(site).geometry().buffer(500)
        
        pred_group = ee.List(ee.List(ee.List(ee.List(el)).get(0)).get(0)) 
        #print(pred_group.getInfo())
        landsat_t01 = ee.List(ee.List(ee.List(ee.List(el)).get(0)).get(0))
        #print(landsat_t01.getInfo())
        modis_t01 = ee.List(ee.List(ee.List(ee.List(el)).get(0)).get(1))
        #print(landsat_t01.getInfo())
        modis_tp = ee.List(ee.List(ee.List(ee.List(el)).get(0)).get(2))
        #print(modis_tp.get(0).getInfo())


        origFine = ee.ImageCollection(ee.List(landsat_t01)).first()   

        # start and end day of group
        doys = landsat_t01 \
            .map(lambda img: ee.String(ee.Image(img).get('DOY')).cat('_'))

        # register images
        landsat_t01, modis_t01, modis_tp = registerImages(landsat_t01,
                                                          modis_t01,
                                                          modis_tp)

        # prep landsat imagery (mask and format)
        maskedLandsat, pixPositions, pixBN = prepLandsat(landsat_t01,
                                                         kernel,
                                                         numPixels,
                                                         commonBandNames,
                                                         doys,
                                                         coverClasses)

        #print('masked landsat', ee.ImageCollection(maskedLandsat).first().geometry().getInfo())


        # prep modis imagery (mask and format)
        modSorted_t01, modSorted_tp = prepMODIS(modis_t01, modis_tp, kernel,
                                                numPixels, commonBandNames,
                                                pixBN)

        #print('modSorted t01_1', ee.ImageCollection(modSorted_t01).first().geometry().getInfo())

        # calculate spectral distance
        specDist = calcSpecDist(maskedLandsat, modSorted_t01,
                                numPixels, pixPositions)

        # calculate spatial distance
        spatDist = calcSpatDist(pixPositions)

        # calculate weights from the spatial and spectral distances
        weights = calcWeight(spatDist, specDist)

        # calculate the conversion coefficients
        coeffs = calcConversionCoeff(maskedLandsat, modSorted_t01,
                                     doys, numPixels, commonBandNames)


        # predict all modis images in modis tp collection
        prediction = ee.List(modSorted_tp) \
            .map(lambda image:
                 predictLandsat(landsat_t01, modSorted_t01,
                                doys, ee.List(image),
                                weights, coeffs,
                                commonBandNames, numPixels)) 
        
        predsColl = ee.ImageCollection(ee.List(prediction)).map(lambda img: img.toFloat())
        
        def waterOnly(img):
            
            img = ee.Image(img)
            
            #Band quality thresholds
            white_pixels = 2000 #white pixels #2500 is max
            
                        
            #create band theshold masks
            bright_pixels = img.select('red').lt(white_pixels).And(img.select('blue').lt(white_pixels))\
                                .And(img.select('green').lt(white_pixels))

            #mask the image collection to keep only water pixels
            #water = gsw.select('occurrence').gte(80);

            road = ee.FeatureCollection('TIGER/2016/Roads').filterBounds(ee.Feature(site).geometry().buffer(250)).geometry().buffer(30)
            roadMask = ee.Image(0).paint(road,1).Not()

            #Collect median elevation for the water pixels where reflectance is extracted
            DEMout = ee.Image("users/eeProject/MERIT").reduceRegion(ee.Reducer.min(), sdist, 30);
            #pixelCount = img.select('red').reduceRegion(ee.Reducer.count(), sdist, 30).get('red')
            
            d = Dswe(img).select('dswe')

            masked = img \
                    .updateMask(roadMask)\
                    .addBands(d).updateMask(d.eq(1).Or(d.eq(2)))\
                    .updateMask(bright_pixels)\
                    .set({'elevation':DEMout.get('elevation')})\
                    .set({'date': img.date().format('YYYY-MM-dd')}).toInt16() ##.updateMask(water).updateMask(land).updateMask(red_positive)\
                    #                    .set({"pixelCount": img.reduceRegion(ee.Reducer.count(), ee.Feature(site).buffer(750).geometry(), 30).get('blue')})\

            

            reducers = ee.Reducer.median().combine(ee.Reducer.stdDev(), "", True)
            previous = ee.List(['blue_stdDev', 'green_stdDev', 'red_stdDev', 'nir_stdDev', 'swir1_stdDev', 
                                'swir2_stdDev',
            'blue_median', 'green_median', 'red_median', 'nir_median', 'swir1_median', 'swir2_median']) #
                #ee.List(['r_gb_median', 'r_gb_stdDev'])
            to = ee.List(['blue_sd', 'green_sd', 'red_sd', 'nir_sd', 'swir1_sd', 'swir2_sd',
             'blue', 'green', 'red', 'nir', 'swir1', 'swir2']) #
            #ee.List(['R_GB', 'R_GB_sd'])

            reduceRegion = masked.reduceRegion(**{'reducer' : reducers, 
                                                  'geometry' : sdist,
                                                  'scale' : 30,
                                                  'crs': origFine.projection(),
                                                  'tileScale': 8}).rename(previous, to)


            output = ee.Feature(site)\
                        .set({"blue": reduceRegion.get('blue')})\
                        .set({"green": reduceRegion.get('green')})\
                        .set({"red": reduceRegion.get('red')})\
                        .set({"nir": reduceRegion.get('nir')})\
                        .set({"swir1": reduceRegion.get('swir1')})\
                        .set({"swir2": reduceRegion.get('swir2')})\
                        .set({"swir1_sd": reduceRegion.get('swir1_sd')})\
                        .set({"blue_sd": reduceRegion.get('blue_sd')})\
                        .set({"green_sd": reduceRegion.get('green_sd')})\
                        .set({"red_sd": reduceRegion.get('red_sd')})\
                        .set({"nir_sd": reduceRegion.get('nir_sd')})\
                        .set({"swir2_sd": reduceRegion.get('swir2_sd')})\
                        .set({"date": masked.get('date')})\
                        .set({"pixelCount": masked.reduceRegion(ee.Reducer.count(), 
                                sdist, 30).get('red')}) #  .set({'dswe': reduceRegion.get('dswe')})\
                        #.set({'dswe_sd': reduceRegion.get('dswe_sd')})\

            return ee.FeatureCollection(output).map(removeGeometry)
        
        
        return ee.ImageCollection(predsColl).map(waterOnly).flatten()  #(ee.List(prediction).map(waterOnly)).flatten()
           
    return ee.FeatureCollection(subsFeat).map(subPred).flatten().filter(ee.Filter.gt('pixelCount', 0))\
                .filter(ee.Filter.gt('swir2', 0))\
                .filter(ee.Filter.gt('red', 0))\
                .filter(ee.Filter.gt('blue', 0))\
                .filter(ee.Filter.gt('green', 0))\
                .filter(ee.Filter.gt('nir', 0))\
                .filter(ee.Filter.gt('swir1', 0))


def featTest(fcList):
    for d in datesStag:
        for i in range(0, fcList.size().getInfo()):
                       
            startDate = str(d[0])
            endDate = str(d[1])
            site = ee.Feature(fcList.get(i))

            #Call sorting and predicting functions
            results = ee.FeatureCollection(site).map(sites_processing(startDate, endDate)).flatten()
            
            task = ee.batch.Export.table.toDrive(
                        collection = ee.FeatureCollection(results), 
                        description = str(i) + '_' + startDate + '-' + endDate + str(huc) + str(siteSpan), 
                        folder = 'MasterTest' + str(huc), 
                        fileFormat = 'csv',
                        selectors = ['SiteID','date',
                                    'blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'pixelCount',
                                    'blue_sd', 'green_sd', 'red_sd', 'nir_sd', 'swir1_sd', 'swir2_sd'])

            #maximum tasks function(max running jobs, waiting period between checking to submit more)
            import time
            maximum_no_of_tasks(2999, 45)

            #Start the task
            task.start()
            print('done', i, startDate, endDate, str(huc), str(siteSpan))
        

Enter verification code: 4/1AbUR2VOXgmwDYPDSon1j-_EUCFsmn91hi4e3YYK1N7lXW9SkJVHbxC1xmCo

Successfully saved authorization token.


In [23]:
###HUC04
#123 Sites
#4/23/23

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc04_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(115,123)
huc = '_huc04'
siteSpan = '_115-123'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc04 _115-123
done 1 2013-03-18 2013-11-01 _huc04 _115-123
done 2 2013-03-18 2013-11-01 _huc04 _115-123
done 3 2013-03-18 2013-11-01 _huc04 _115-123
done 4 2013-03-18 2013-11-01 _huc04 _115-123
done 5 2013-03-18 2013-11-01 _huc04 _115-123
done 6 2013-03-18 2013-11-01 _huc04 _115-123
done 7 2013-03-18 2013-11-01 _huc04 _115-123
done 0 2013-10-01 2014-05-01 _huc04 _115-123
done 1 2013-10-01 2014-05-01 _huc04 _115-123
done 2 2013-10-01 2014-05-01 _huc04 _115-123
done 3 2013-10-01 2014-05-01 _huc04 _115-123
done 4 2013-10-01 2014-05-01 _huc04 _115-123
done 5 2013-10-01 2014-05-01 _huc04 _115-123
done 6 2013-10-01 2014-05-01 _huc04 _115-123
done 7 2013-10-01 2014-05-01 _huc04 _115-123
done 0 2014-04-01 2014-11-01 _huc04 _115-123
done 1 2014-04-01 2014-11-01 _huc04 _115-123
done 2 2014-04-01 2014-11-01 _huc04 _115-123
done 3 2014-04-01 2014-11-01 _huc04 _115-123
done 4 2014-04-01 2014-11-01 _huc04 _115-123
done 5 2014-04-01 2014-11-01 _huc04 _115-123
done 6 201

In [21]:
###HUC04
#123 Sites
#4/23/23

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc04_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(110,115)
huc = '_huc04'
siteSpan = '_110-115'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc04 _110-115
done 1 2013-03-18 2013-11-01 _huc04 _110-115
done 2 2013-03-18 2013-11-01 _huc04 _110-115
done 3 2013-03-18 2013-11-01 _huc04 _110-115
done 4 2013-03-18 2013-11-01 _huc04 _110-115
done 0 2013-10-01 2014-05-01 _huc04 _110-115
done 1 2013-10-01 2014-05-01 _huc04 _110-115
done 2 2013-10-01 2014-05-01 _huc04 _110-115
done 3 2013-10-01 2014-05-01 _huc04 _110-115
done 4 2013-10-01 2014-05-01 _huc04 _110-115
done 0 2014-04-01 2014-11-01 _huc04 _110-115
done 1 2014-04-01 2014-11-01 _huc04 _110-115
done 2 2014-04-01 2014-11-01 _huc04 _110-115
done 3 2014-04-01 2014-11-01 _huc04 _110-115
done 4 2014-04-01 2014-11-01 _huc04 _110-115
done 0 2014-10-01 2015-05-01 _huc04 _110-115
done 1 2014-10-01 2015-05-01 _huc04 _110-115
done 2 2014-10-01 2015-05-01 _huc04 _110-115
done 3 2014-10-01 2015-05-01 _huc04 _110-115
done 4 2014-10-01 2015-05-01 _huc04 _110-115
done 0 2015-04-01 2015-11-01 _huc04 _110-115
done 1 2015-04-01 2015-11-01 _huc04 _110-115
done 2 201

In [19]:
###HUC04
#123 Sites
#4/23/23

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc04_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(100,110)
huc = '_huc04'
siteSpan = '_100-110'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc04 _100-110
done 1 2013-03-18 2013-11-01 _huc04 _100-110
done 2 2013-03-18 2013-11-01 _huc04 _100-110
done 3 2013-03-18 2013-11-01 _huc04 _100-110
done 4 2013-03-18 2013-11-01 _huc04 _100-110
done 5 2013-03-18 2013-11-01 _huc04 _100-110
done 6 2013-03-18 2013-11-01 _huc04 _100-110
done 7 2013-03-18 2013-11-01 _huc04 _100-110
done 8 2013-03-18 2013-11-01 _huc04 _100-110
done 9 2013-03-18 2013-11-01 _huc04 _100-110
done 0 2013-10-01 2014-05-01 _huc04 _100-110
done 1 2013-10-01 2014-05-01 _huc04 _100-110
done 2 2013-10-01 2014-05-01 _huc04 _100-110
done 3 2013-10-01 2014-05-01 _huc04 _100-110
done 4 2013-10-01 2014-05-01 _huc04 _100-110
done 5 2013-10-01 2014-05-01 _huc04 _100-110
done 6 2013-10-01 2014-05-01 _huc04 _100-110
done 7 2013-10-01 2014-05-01 _huc04 _100-110
done 8 2013-10-01 2014-05-01 _huc04 _100-110
done 9 2013-10-01 2014-05-01 _huc04 _100-110
done 0 2014-04-01 2014-11-01 _huc04 _100-110
done 1 2014-04-01 2014-11-01 _huc04 _100-110
done 2 201

done 3 2022-04-01 2022-11-01 _huc04 _100-110
done 4 2022-04-01 2022-11-01 _huc04 _100-110
done 5 2022-04-01 2022-11-01 _huc04 _100-110
done 6 2022-04-01 2022-11-01 _huc04 _100-110
done 7 2022-04-01 2022-11-01 _huc04 _100-110
done 8 2022-04-01 2022-11-01 _huc04 _100-110
done 9 2022-04-01 2022-11-01 _huc04 _100-110
done 0 2022-10-01 2023-03-01 _huc04 _100-110
done 1 2022-10-01 2023-03-01 _huc04 _100-110
done 2 2022-10-01 2023-03-01 _huc04 _100-110
done 3 2022-10-01 2023-03-01 _huc04 _100-110
done 4 2022-10-01 2023-03-01 _huc04 _100-110
done 5 2022-10-01 2023-03-01 _huc04 _100-110
done 6 2022-10-01 2023-03-01 _huc04 _100-110
done 7 2022-10-01 2023-03-01 _huc04 _100-110
done 8 2022-10-01 2023-03-01 _huc04 _100-110
done 9 2022-10-01 2023-03-01 _huc04 _100-110


In [17]:
###HUC04
#123 Sites
#4/23/23

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc04_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(90,100)
huc = '_huc04'
siteSpan = '_90-100'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc04 _90-100
done 1 2013-03-18 2013-11-01 _huc04 _90-100
done 2 2013-03-18 2013-11-01 _huc04 _90-100
done 3 2013-03-18 2013-11-01 _huc04 _90-100
done 4 2013-03-18 2013-11-01 _huc04 _90-100
done 5 2013-03-18 2013-11-01 _huc04 _90-100
done 6 2013-03-18 2013-11-01 _huc04 _90-100
done 7 2013-03-18 2013-11-01 _huc04 _90-100
done 8 2013-03-18 2013-11-01 _huc04 _90-100
done 9 2013-03-18 2013-11-01 _huc04 _90-100
done 0 2013-10-01 2014-05-01 _huc04 _90-100
done 1 2013-10-01 2014-05-01 _huc04 _90-100
done 2 2013-10-01 2014-05-01 _huc04 _90-100
done 3 2013-10-01 2014-05-01 _huc04 _90-100
done 4 2013-10-01 2014-05-01 _huc04 _90-100
done 5 2013-10-01 2014-05-01 _huc04 _90-100
done 6 2013-10-01 2014-05-01 _huc04 _90-100
done 7 2013-10-01 2014-05-01 _huc04 _90-100
done 8 2013-10-01 2014-05-01 _huc04 _90-100
done 9 2013-10-01 2014-05-01 _huc04 _90-100
done 0 2014-04-01 2014-11-01 _huc04 _90-100
done 1 2014-04-01 2014-11-01 _huc04 _90-100
done 2 2014-04-01 2014-11-01 _hu

done 7 2022-04-01 2022-11-01 _huc04 _90-100
done 8 2022-04-01 2022-11-01 _huc04 _90-100
done 9 2022-04-01 2022-11-01 _huc04 _90-100
done 0 2022-10-01 2023-03-01 _huc04 _90-100
done 1 2022-10-01 2023-03-01 _huc04 _90-100
done 2 2022-10-01 2023-03-01 _huc04 _90-100
done 3 2022-10-01 2023-03-01 _huc04 _90-100
done 4 2022-10-01 2023-03-01 _huc04 _90-100
done 5 2022-10-01 2023-03-01 _huc04 _90-100
done 6 2022-10-01 2023-03-01 _huc04 _90-100
done 7 2022-10-01 2023-03-01 _huc04 _90-100
done 8 2022-10-01 2023-03-01 _huc04 _90-100
done 9 2022-10-01 2023-03-01 _huc04 _90-100


In [15]:
###HUC04
#123 Sites
#4/23/23
datesStag = (('2015-10-01', '2016-05-01'),
        ('2016-04-01', '2016-11-01'),
        ('2016-10-01', '2017-05-01'),
        ('2017-04-01', '2017-11-01'),
        ('2017-10-01', '2018-05-01'),
        ('2018-04-01', '2018-11-01'),
        ('2018-10-01', '2019-05-01'),
        ('2019-04-01', '2019-11-01'),
        ('2019-10-01', '2020-05-01'),
        ('2020-04-01', '2020-11-01'),
        ('2020-10-01', '2021-05-01'),
        ('2021-04-01', '2021-11-01'),
        ('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc04_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(80,90)
huc = '_huc04'
siteSpan = '_80-90'

#Call function
featTest(hucList)

done 0 2015-10-01 2016-05-01 _huc04 _80-90
done 1 2015-10-01 2016-05-01 _huc04 _80-90
done 2 2015-10-01 2016-05-01 _huc04 _80-90
done 3 2015-10-01 2016-05-01 _huc04 _80-90
done 4 2015-10-01 2016-05-01 _huc04 _80-90
done 5 2015-10-01 2016-05-01 _huc04 _80-90
done 6 2015-10-01 2016-05-01 _huc04 _80-90
done 7 2015-10-01 2016-05-01 _huc04 _80-90
done 8 2015-10-01 2016-05-01 _huc04 _80-90
done 9 2015-10-01 2016-05-01 _huc04 _80-90
done 0 2016-04-01 2016-11-01 _huc04 _80-90
done 1 2016-04-01 2016-11-01 _huc04 _80-90
done 2 2016-04-01 2016-11-01 _huc04 _80-90
done 3 2016-04-01 2016-11-01 _huc04 _80-90
done 4 2016-04-01 2016-11-01 _huc04 _80-90
done 5 2016-04-01 2016-11-01 _huc04 _80-90
done 6 2016-04-01 2016-11-01 _huc04 _80-90
done 7 2016-04-01 2016-11-01 _huc04 _80-90
done 8 2016-04-01 2016-11-01 _huc04 _80-90
done 9 2016-04-01 2016-11-01 _huc04 _80-90
done 0 2016-10-01 2017-05-01 _huc04 _80-90
done 1 2016-10-01 2017-05-01 _huc04 _80-90
done 2 2016-10-01 2017-05-01 _huc04 _80-90
done 3 2016

In [12]:
###HUC04
#123 Sites
#4/23/23


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc04_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(70,80)
huc = '_huc04'
siteSpan = '_70-80'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc04 _70-80
done 1 2013-03-18 2013-11-01 _huc04 _70-80
done 2 2013-03-18 2013-11-01 _huc04 _70-80
done 3 2013-03-18 2013-11-01 _huc04 _70-80
done 4 2013-03-18 2013-11-01 _huc04 _70-80
done 5 2013-03-18 2013-11-01 _huc04 _70-80
done 6 2013-03-18 2013-11-01 _huc04 _70-80
done 7 2013-03-18 2013-11-01 _huc04 _70-80
done 8 2013-03-18 2013-11-01 _huc04 _70-80
done 9 2013-03-18 2013-11-01 _huc04 _70-80
done 0 2013-10-01 2014-05-01 _huc04 _70-80
done 1 2013-10-01 2014-05-01 _huc04 _70-80
done 2 2013-10-01 2014-05-01 _huc04 _70-80
done 3 2013-10-01 2014-05-01 _huc04 _70-80
done 4 2013-10-01 2014-05-01 _huc04 _70-80
done 5 2013-10-01 2014-05-01 _huc04 _70-80
done 6 2013-10-01 2014-05-01 _huc04 _70-80
done 7 2013-10-01 2014-05-01 _huc04 _70-80
done 8 2013-10-01 2014-05-01 _huc04 _70-80
done 9 2013-10-01 2014-05-01 _huc04 _70-80
done 0 2014-04-01 2014-11-01 _huc04 _70-80
done 1 2014-04-01 2014-11-01 _huc04 _70-80
done 2 2014-04-01 2014-11-01 _huc04 _70-80
done 3 2014

done 1 2022-10-01 2023-03-01 _huc04 _70-80
done 2 2022-10-01 2023-03-01 _huc04 _70-80
done 3 2022-10-01 2023-03-01 _huc04 _70-80
done 4 2022-10-01 2023-03-01 _huc04 _70-80
done 5 2022-10-01 2023-03-01 _huc04 _70-80
done 6 2022-10-01 2023-03-01 _huc04 _70-80
done 7 2022-10-01 2023-03-01 _huc04 _70-80
done 8 2022-10-01 2023-03-01 _huc04 _70-80
done 9 2022-10-01 2023-03-01 _huc04 _70-80


In [10]:
###HUC04
#123 Sites
#4/23/23


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc04_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(60,70)
huc = '_huc04'
siteSpan = '_60-70'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc04 _60-70
done 1 2013-03-18 2013-11-01 _huc04 _60-70
done 2 2013-03-18 2013-11-01 _huc04 _60-70
done 3 2013-03-18 2013-11-01 _huc04 _60-70
done 4 2013-03-18 2013-11-01 _huc04 _60-70
done 5 2013-03-18 2013-11-01 _huc04 _60-70
done 6 2013-03-18 2013-11-01 _huc04 _60-70
done 7 2013-03-18 2013-11-01 _huc04 _60-70
done 8 2013-03-18 2013-11-01 _huc04 _60-70
done 9 2013-03-18 2013-11-01 _huc04 _60-70
done 0 2013-10-01 2014-05-01 _huc04 _60-70
done 1 2013-10-01 2014-05-01 _huc04 _60-70
done 2 2013-10-01 2014-05-01 _huc04 _60-70
done 3 2013-10-01 2014-05-01 _huc04 _60-70
done 4 2013-10-01 2014-05-01 _huc04 _60-70
done 5 2013-10-01 2014-05-01 _huc04 _60-70
done 6 2013-10-01 2014-05-01 _huc04 _60-70
done 7 2013-10-01 2014-05-01 _huc04 _60-70
done 8 2013-10-01 2014-05-01 _huc04 _60-70
done 9 2013-10-01 2014-05-01 _huc04 _60-70
done 0 2014-04-01 2014-11-01 _huc04 _60-70
done 1 2014-04-01 2014-11-01 _huc04 _60-70
done 2 2014-04-01 2014-11-01 _huc04 _60-70
done 3 2014

done 1 2022-10-01 2023-03-01 _huc04 _60-70
done 2 2022-10-01 2023-03-01 _huc04 _60-70
done 3 2022-10-01 2023-03-01 _huc04 _60-70
done 4 2022-10-01 2023-03-01 _huc04 _60-70
done 5 2022-10-01 2023-03-01 _huc04 _60-70
done 6 2022-10-01 2023-03-01 _huc04 _60-70
done 7 2022-10-01 2023-03-01 _huc04 _60-70
done 8 2022-10-01 2023-03-01 _huc04 _60-70
done 9 2022-10-01 2023-03-01 _huc04 _60-70


In [8]:
###HUC04
#123 Sites
#4/23/23


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc04_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(50,60)
huc = '_huc04'
siteSpan = '_50-60'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc04 _50-60
done 1 2013-03-18 2013-11-01 _huc04 _50-60
done 2 2013-03-18 2013-11-01 _huc04 _50-60
done 3 2013-03-18 2013-11-01 _huc04 _50-60
done 4 2013-03-18 2013-11-01 _huc04 _50-60
done 5 2013-03-18 2013-11-01 _huc04 _50-60
done 6 2013-03-18 2013-11-01 _huc04 _50-60
done 7 2013-03-18 2013-11-01 _huc04 _50-60
done 8 2013-03-18 2013-11-01 _huc04 _50-60
done 9 2013-03-18 2013-11-01 _huc04 _50-60
done 0 2013-10-01 2014-05-01 _huc04 _50-60
done 1 2013-10-01 2014-05-01 _huc04 _50-60
done 2 2013-10-01 2014-05-01 _huc04 _50-60
done 3 2013-10-01 2014-05-01 _huc04 _50-60
done 4 2013-10-01 2014-05-01 _huc04 _50-60
done 5 2013-10-01 2014-05-01 _huc04 _50-60
done 6 2013-10-01 2014-05-01 _huc04 _50-60
done 7 2013-10-01 2014-05-01 _huc04 _50-60
done 8 2013-10-01 2014-05-01 _huc04 _50-60
done 9 2013-10-01 2014-05-01 _huc04 _50-60
done 0 2014-04-01 2014-11-01 _huc04 _50-60
done 1 2014-04-01 2014-11-01 _huc04 _50-60
done 2 2014-04-01 2014-11-01 _huc04 _50-60
done 3 2014

done 1 2022-10-01 2023-03-01 _huc04 _50-60
done 2 2022-10-01 2023-03-01 _huc04 _50-60
done 3 2022-10-01 2023-03-01 _huc04 _50-60
done 4 2022-10-01 2023-03-01 _huc04 _50-60
done 5 2022-10-01 2023-03-01 _huc04 _50-60
done 6 2022-10-01 2023-03-01 _huc04 _50-60
done 7 2022-10-01 2023-03-01 _huc04 _50-60
done 8 2022-10-01 2023-03-01 _huc04 _50-60
done 9 2022-10-01 2023-03-01 _huc04 _50-60


In [6]:
###HUC04
#123 Sites
#4/23/23

datesStag = (('2016-10-01', '2017-05-01'),
        ('2017-04-01', '2017-11-01'),
        ('2017-10-01', '2018-05-01'),
        ('2018-04-01', '2018-11-01'),
        ('2018-10-01', '2019-05-01'),
        ('2019-04-01', '2019-11-01'),
        ('2019-10-01', '2020-05-01'),
        ('2020-04-01', '2020-11-01'),
        ('2020-10-01', '2021-05-01'),
        ('2021-04-01', '2021-11-01'),
        ('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))
#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc04_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(40,50)
huc = '_huc04'
siteSpan = '_40-50'

#Call function
featTest(hucList)

done 0 2016-10-01 2017-05-01 _huc04 _40-50
done 1 2016-10-01 2017-05-01 _huc04 _40-50
done 2 2016-10-01 2017-05-01 _huc04 _40-50
done 3 2016-10-01 2017-05-01 _huc04 _40-50
done 4 2016-10-01 2017-05-01 _huc04 _40-50
done 5 2016-10-01 2017-05-01 _huc04 _40-50
done 6 2016-10-01 2017-05-01 _huc04 _40-50
done 7 2016-10-01 2017-05-01 _huc04 _40-50
done 8 2016-10-01 2017-05-01 _huc04 _40-50
done 9 2016-10-01 2017-05-01 _huc04 _40-50
done 0 2017-04-01 2017-11-01 _huc04 _40-50
done 1 2017-04-01 2017-11-01 _huc04 _40-50
done 2 2017-04-01 2017-11-01 _huc04 _40-50
done 3 2017-04-01 2017-11-01 _huc04 _40-50
done 4 2017-04-01 2017-11-01 _huc04 _40-50
done 5 2017-04-01 2017-11-01 _huc04 _40-50
done 6 2017-04-01 2017-11-01 _huc04 _40-50
done 7 2017-04-01 2017-11-01 _huc04 _40-50
done 8 2017-04-01 2017-11-01 _huc04 _40-50
done 9 2017-04-01 2017-11-01 _huc04 _40-50
done 0 2017-10-01 2018-05-01 _huc04 _40-50
done 1 2017-10-01 2018-05-01 _huc04 _40-50
done 2 2017-10-01 2018-05-01 _huc04 _40-50
done 3 2017

In [22]:
###HUC04
#123 Sites
#4/23/23

datesStag = (('2017-10-01', '2018-05-01'),
        ('2018-04-01', '2018-11-01'),
        ('2018-10-01', '2019-05-01'),
        ('2019-04-01', '2019-11-01'),
        ('2019-10-01', '2020-05-01'),
        ('2020-04-01', '2020-11-01'),
        ('2020-10-01', '2021-05-01'),
        ('2021-04-01', '2021-11-01'),
        ('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc04_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(30,40)
huc = '_huc04'
siteSpan = '_30-40'

#Call function
featTest(hucList)

done 0 2017-10-01 2018-05-01 _huc04 _30-40
done 1 2017-10-01 2018-05-01 _huc04 _30-40
done 2 2017-10-01 2018-05-01 _huc04 _30-40
done 3 2017-10-01 2018-05-01 _huc04 _30-40
done 4 2017-10-01 2018-05-01 _huc04 _30-40
done 5 2017-10-01 2018-05-01 _huc04 _30-40
done 6 2017-10-01 2018-05-01 _huc04 _30-40
done 7 2017-10-01 2018-05-01 _huc04 _30-40
done 8 2017-10-01 2018-05-01 _huc04 _30-40
done 9 2017-10-01 2018-05-01 _huc04 _30-40
done 0 2018-04-01 2018-11-01 _huc04 _30-40
done 1 2018-04-01 2018-11-01 _huc04 _30-40
done 2 2018-04-01 2018-11-01 _huc04 _30-40
done 3 2018-04-01 2018-11-01 _huc04 _30-40
done 4 2018-04-01 2018-11-01 _huc04 _30-40
done 5 2018-04-01 2018-11-01 _huc04 _30-40
done 6 2018-04-01 2018-11-01 _huc04 _30-40
done 7 2018-04-01 2018-11-01 _huc04 _30-40
done 8 2018-04-01 2018-11-01 _huc04 _30-40
done 9 2018-04-01 2018-11-01 _huc04 _30-40
done 0 2018-10-01 2019-05-01 _huc04 _30-40
done 1 2018-10-01 2019-05-01 _huc04 _30-40
done 2 2018-10-01 2019-05-01 _huc04 _30-40
done 3 2018

In [18]:
###HUC04
#123 Sites
#4/23/23


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc04_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(20,30)
huc = '_huc04'
siteSpan = '_20-30'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc04 _20-30
done 1 2013-03-18 2013-11-01 _huc04 _20-30
done 2 2013-03-18 2013-11-01 _huc04 _20-30
done 3 2013-03-18 2013-11-01 _huc04 _20-30
done 4 2013-03-18 2013-11-01 _huc04 _20-30
done 5 2013-03-18 2013-11-01 _huc04 _20-30
done 6 2013-03-18 2013-11-01 _huc04 _20-30
done 7 2013-03-18 2013-11-01 _huc04 _20-30
done 8 2013-03-18 2013-11-01 _huc04 _20-30
done 9 2013-03-18 2013-11-01 _huc04 _20-30
done 0 2013-10-01 2014-05-01 _huc04 _20-30
done 1 2013-10-01 2014-05-01 _huc04 _20-30
done 2 2013-10-01 2014-05-01 _huc04 _20-30
done 3 2013-10-01 2014-05-01 _huc04 _20-30
done 4 2013-10-01 2014-05-01 _huc04 _20-30
done 5 2013-10-01 2014-05-01 _huc04 _20-30
done 6 2013-10-01 2014-05-01 _huc04 _20-30
done 7 2013-10-01 2014-05-01 _huc04 _20-30
done 8 2013-10-01 2014-05-01 _huc04 _20-30
done 9 2013-10-01 2014-05-01 _huc04 _20-30
done 0 2014-04-01 2014-11-01 _huc04 _20-30
done 1 2014-04-01 2014-11-01 _huc04 _20-30
done 2 2014-04-01 2014-11-01 _huc04 _20-30
done 3 2014

done 1 2022-10-01 2023-03-01 _huc04 _20-30
done 2 2022-10-01 2023-03-01 _huc04 _20-30
done 3 2022-10-01 2023-03-01 _huc04 _20-30
done 4 2022-10-01 2023-03-01 _huc04 _20-30
done 5 2022-10-01 2023-03-01 _huc04 _20-30
done 6 2022-10-01 2023-03-01 _huc04 _20-30
done 7 2022-10-01 2023-03-01 _huc04 _20-30
done 8 2022-10-01 2023-03-01 _huc04 _20-30
done 9 2022-10-01 2023-03-01 _huc04 _20-30


In [16]:
###HUC04
#123 Sites
#4/23/23


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc04_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(10,20)
huc = '_huc04'
siteSpan = '_10-20'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc04 _10-20
done 1 2013-03-18 2013-11-01 _huc04 _10-20
done 2 2013-03-18 2013-11-01 _huc04 _10-20
done 3 2013-03-18 2013-11-01 _huc04 _10-20
done 4 2013-03-18 2013-11-01 _huc04 _10-20
done 5 2013-03-18 2013-11-01 _huc04 _10-20
done 6 2013-03-18 2013-11-01 _huc04 _10-20
done 7 2013-03-18 2013-11-01 _huc04 _10-20
done 8 2013-03-18 2013-11-01 _huc04 _10-20
done 9 2013-03-18 2013-11-01 _huc04 _10-20
done 0 2013-10-01 2014-05-01 _huc04 _10-20
done 1 2013-10-01 2014-05-01 _huc04 _10-20
done 2 2013-10-01 2014-05-01 _huc04 _10-20
done 3 2013-10-01 2014-05-01 _huc04 _10-20
done 4 2013-10-01 2014-05-01 _huc04 _10-20
done 5 2013-10-01 2014-05-01 _huc04 _10-20
done 6 2013-10-01 2014-05-01 _huc04 _10-20
done 7 2013-10-01 2014-05-01 _huc04 _10-20
done 8 2013-10-01 2014-05-01 _huc04 _10-20
done 9 2013-10-01 2014-05-01 _huc04 _10-20
done 0 2014-04-01 2014-11-01 _huc04 _10-20
done 1 2014-04-01 2014-11-01 _huc04 _10-20
done 2 2014-04-01 2014-11-01 _huc04 _10-20
done 3 2014

done 1 2022-10-01 2023-03-01 _huc04 _10-20
done 2 2022-10-01 2023-03-01 _huc04 _10-20
done 3 2022-10-01 2023-03-01 _huc04 _10-20
done 4 2022-10-01 2023-03-01 _huc04 _10-20
done 5 2022-10-01 2023-03-01 _huc04 _10-20
done 6 2022-10-01 2023-03-01 _huc04 _10-20
done 7 2022-10-01 2023-03-01 _huc04 _10-20
done 8 2022-10-01 2023-03-01 _huc04 _10-20
done 9 2022-10-01 2023-03-01 _huc04 _10-20


In [14]:
###HUC04
#123 Sites
#4/23/23


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc04_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(0,10)
huc = '_huc04'
siteSpan = '_0-10'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc04 _0-10
done 1 2013-03-18 2013-11-01 _huc04 _0-10
done 2 2013-03-18 2013-11-01 _huc04 _0-10
done 3 2013-03-18 2013-11-01 _huc04 _0-10
done 4 2013-03-18 2013-11-01 _huc04 _0-10
done 5 2013-03-18 2013-11-01 _huc04 _0-10
done 6 2013-03-18 2013-11-01 _huc04 _0-10
done 7 2013-03-18 2013-11-01 _huc04 _0-10
done 8 2013-03-18 2013-11-01 _huc04 _0-10
done 9 2013-03-18 2013-11-01 _huc04 _0-10
done 0 2013-10-01 2014-05-01 _huc04 _0-10
done 1 2013-10-01 2014-05-01 _huc04 _0-10
done 2 2013-10-01 2014-05-01 _huc04 _0-10
done 3 2013-10-01 2014-05-01 _huc04 _0-10
done 4 2013-10-01 2014-05-01 _huc04 _0-10
done 5 2013-10-01 2014-05-01 _huc04 _0-10
done 6 2013-10-01 2014-05-01 _huc04 _0-10
done 7 2013-10-01 2014-05-01 _huc04 _0-10
done 8 2013-10-01 2014-05-01 _huc04 _0-10
done 9 2013-10-01 2014-05-01 _huc04 _0-10
done 0 2014-04-01 2014-11-01 _huc04 _0-10
done 1 2014-04-01 2014-11-01 _huc04 _0-10
done 2 2014-04-01 2014-11-01 _huc04 _0-10
done 3 2014-04-01 2014-11-01 _huc0

done 6 2022-10-01 2023-03-01 _huc04 _0-10
done 7 2022-10-01 2023-03-01 _huc04 _0-10
done 8 2022-10-01 2023-03-01 _huc04 _0-10
done 9 2022-10-01 2023-03-01 _huc04 _0-10


In [12]:
###HUC03
#129 Sites
#4/18/23


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc03_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(125,129)
huc = '_huc03'
siteSpan = '_125-129'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc03 _125-129
done 1 2013-03-18 2013-11-01 _huc03 _125-129
done 2 2013-03-18 2013-11-01 _huc03 _125-129
done 3 2013-03-18 2013-11-01 _huc03 _125-129
done 0 2013-10-01 2014-05-01 _huc03 _125-129
done 1 2013-10-01 2014-05-01 _huc03 _125-129
done 2 2013-10-01 2014-05-01 _huc03 _125-129
done 3 2013-10-01 2014-05-01 _huc03 _125-129
done 0 2014-04-01 2014-11-01 _huc03 _125-129
done 1 2014-04-01 2014-11-01 _huc03 _125-129
done 2 2014-04-01 2014-11-01 _huc03 _125-129
done 3 2014-04-01 2014-11-01 _huc03 _125-129
done 0 2014-10-01 2015-05-01 _huc03 _125-129
done 1 2014-10-01 2015-05-01 _huc03 _125-129
done 2 2014-10-01 2015-05-01 _huc03 _125-129
done 3 2014-10-01 2015-05-01 _huc03 _125-129
done 0 2015-04-01 2015-11-01 _huc03 _125-129
done 1 2015-04-01 2015-11-01 _huc03 _125-129
done 2 2015-04-01 2015-11-01 _huc03 _125-129
done 3 2015-04-01 2015-11-01 _huc03 _125-129
done 0 2015-10-01 2016-05-01 _huc03 _125-129
done 1 2015-10-01 2016-05-01 _huc03 _125-129
done 2 201

In [11]:
###HUC03
#129 Sites
#4/18/23


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc03_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(120,125)
huc = '_huc03'
siteSpan = '_120-125'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc03 _120-125
done 1 2013-03-18 2013-11-01 _huc03 _120-125
done 2 2013-03-18 2013-11-01 _huc03 _120-125
done 3 2013-03-18 2013-11-01 _huc03 _120-125
done 4 2013-03-18 2013-11-01 _huc03 _120-125
done 0 2013-10-01 2014-05-01 _huc03 _120-125
done 1 2013-10-01 2014-05-01 _huc03 _120-125
done 2 2013-10-01 2014-05-01 _huc03 _120-125
done 3 2013-10-01 2014-05-01 _huc03 _120-125
done 4 2013-10-01 2014-05-01 _huc03 _120-125
done 0 2014-04-01 2014-11-01 _huc03 _120-125
done 1 2014-04-01 2014-11-01 _huc03 _120-125
done 2 2014-04-01 2014-11-01 _huc03 _120-125
done 3 2014-04-01 2014-11-01 _huc03 _120-125
done 4 2014-04-01 2014-11-01 _huc03 _120-125
done 0 2014-10-01 2015-05-01 _huc03 _120-125
done 1 2014-10-01 2015-05-01 _huc03 _120-125
done 2 2014-10-01 2015-05-01 _huc03 _120-125
done 3 2014-10-01 2015-05-01 _huc03 _120-125
done 4 2014-10-01 2015-05-01 _huc03 _120-125
done 0 2015-04-01 2015-11-01 _huc03 _120-125
done 1 2015-04-01 2015-11-01 _huc03 _120-125
done 2 201

In [10]:
###HUC03
#129 Sites
#4/18/23


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc03_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(115,120)
huc = '_huc03'
siteSpan = '_115-120'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc03 _115-120
done 1 2013-03-18 2013-11-01 _huc03 _115-120
done 2 2013-03-18 2013-11-01 _huc03 _115-120
done 3 2013-03-18 2013-11-01 _huc03 _115-120
done 4 2013-03-18 2013-11-01 _huc03 _115-120
done 0 2013-10-01 2014-05-01 _huc03 _115-120
done 1 2013-10-01 2014-05-01 _huc03 _115-120
done 2 2013-10-01 2014-05-01 _huc03 _115-120
done 3 2013-10-01 2014-05-01 _huc03 _115-120
done 4 2013-10-01 2014-05-01 _huc03 _115-120
done 0 2014-04-01 2014-11-01 _huc03 _115-120
done 1 2014-04-01 2014-11-01 _huc03 _115-120
done 2 2014-04-01 2014-11-01 _huc03 _115-120
done 3 2014-04-01 2014-11-01 _huc03 _115-120
done 4 2014-04-01 2014-11-01 _huc03 _115-120
done 0 2014-10-01 2015-05-01 _huc03 _115-120
done 1 2014-10-01 2015-05-01 _huc03 _115-120
done 2 2014-10-01 2015-05-01 _huc03 _115-120
done 3 2014-10-01 2015-05-01 _huc03 _115-120
done 4 2014-10-01 2015-05-01 _huc03 _115-120
done 0 2015-04-01 2015-11-01 _huc03 _115-120
done 1 2015-04-01 2015-11-01 _huc03 _115-120
done 2 201

In [8]:
###HUC03
#129 Sites
#4/18/23


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc03_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(110,115)
huc = '_huc03'
siteSpan = '_110-115'

#Call function
featTest(hucList)

KeyboardInterrupt: 

In [4]:
###HUC03
#129 Sites
#4/18/23


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc03_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(100,110)
huc = '_huc03'
siteSpan = '_100-110'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc03 _100-110
done 1 2013-03-18 2013-11-01 _huc03 _100-110
done 2 2013-03-18 2013-11-01 _huc03 _100-110
done 3 2013-03-18 2013-11-01 _huc03 _100-110
done 4 2013-03-18 2013-11-01 _huc03 _100-110
done 5 2013-03-18 2013-11-01 _huc03 _100-110
done 6 2013-03-18 2013-11-01 _huc03 _100-110
done 7 2013-03-18 2013-11-01 _huc03 _100-110
done 8 2013-03-18 2013-11-01 _huc03 _100-110
done 9 2013-03-18 2013-11-01 _huc03 _100-110
done 0 2013-10-01 2014-05-01 _huc03 _100-110
done 1 2013-10-01 2014-05-01 _huc03 _100-110
done 2 2013-10-01 2014-05-01 _huc03 _100-110
done 3 2013-10-01 2014-05-01 _huc03 _100-110
done 4 2013-10-01 2014-05-01 _huc03 _100-110
done 5 2013-10-01 2014-05-01 _huc03 _100-110
done 6 2013-10-01 2014-05-01 _huc03 _100-110
done 7 2013-10-01 2014-05-01 _huc03 _100-110
done 8 2013-10-01 2014-05-01 _huc03 _100-110
done 9 2013-10-01 2014-05-01 _huc03 _100-110
done 0 2014-04-01 2014-11-01 _huc03 _100-110
done 1 2014-04-01 2014-11-01 _huc03 _100-110
done 2 201

done 3 2022-04-01 2022-11-01 _huc03 _100-110
done 4 2022-04-01 2022-11-01 _huc03 _100-110
done 5 2022-04-01 2022-11-01 _huc03 _100-110
done 6 2022-04-01 2022-11-01 _huc03 _100-110
done 7 2022-04-01 2022-11-01 _huc03 _100-110
done 8 2022-04-01 2022-11-01 _huc03 _100-110
done 9 2022-04-01 2022-11-01 _huc03 _100-110
done 0 2022-10-01 2023-03-01 _huc03 _100-110
done 1 2022-10-01 2023-03-01 _huc03 _100-110
done 2 2022-10-01 2023-03-01 _huc03 _100-110
done 3 2022-10-01 2023-03-01 _huc03 _100-110
done 4 2022-10-01 2023-03-01 _huc03 _100-110
done 5 2022-10-01 2023-03-01 _huc03 _100-110
done 6 2022-10-01 2023-03-01 _huc03 _100-110
done 7 2022-10-01 2023-03-01 _huc03 _100-110
done 8 2022-10-01 2023-03-01 _huc03 _100-110
done 9 2022-10-01 2023-03-01 _huc03 _100-110


In [2]:
###HUC03
#129 Sites
#4/18/23


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc03_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(90,100)
huc = '_huc03'
siteSpan = '_90-100'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc03 _90-100
done 1 2013-03-18 2013-11-01 _huc03 _90-100
done 2 2013-03-18 2013-11-01 _huc03 _90-100
done 3 2013-03-18 2013-11-01 _huc03 _90-100
done 4 2013-03-18 2013-11-01 _huc03 _90-100
done 5 2013-03-18 2013-11-01 _huc03 _90-100
done 6 2013-03-18 2013-11-01 _huc03 _90-100
done 7 2013-03-18 2013-11-01 _huc03 _90-100
done 8 2013-03-18 2013-11-01 _huc03 _90-100
done 9 2013-03-18 2013-11-01 _huc03 _90-100
done 0 2013-10-01 2014-05-01 _huc03 _90-100
done 1 2013-10-01 2014-05-01 _huc03 _90-100
done 2 2013-10-01 2014-05-01 _huc03 _90-100
done 3 2013-10-01 2014-05-01 _huc03 _90-100
done 4 2013-10-01 2014-05-01 _huc03 _90-100
done 5 2013-10-01 2014-05-01 _huc03 _90-100
done 6 2013-10-01 2014-05-01 _huc03 _90-100
done 7 2013-10-01 2014-05-01 _huc03 _90-100
done 8 2013-10-01 2014-05-01 _huc03 _90-100
done 9 2013-10-01 2014-05-01 _huc03 _90-100
done 0 2014-04-01 2014-11-01 _huc03 _90-100
done 1 2014-04-01 2014-11-01 _huc03 _90-100
done 2 2014-04-01 2014-11-01 _hu

done 7 2022-04-01 2022-11-01 _huc03 _90-100
done 8 2022-04-01 2022-11-01 _huc03 _90-100
done 9 2022-04-01 2022-11-01 _huc03 _90-100
done 0 2022-10-01 2023-03-01 _huc03 _90-100
done 1 2022-10-01 2023-03-01 _huc03 _90-100
done 2 2022-10-01 2023-03-01 _huc03 _90-100
done 3 2022-10-01 2023-03-01 _huc03 _90-100
done 4 2022-10-01 2023-03-01 _huc03 _90-100
done 5 2022-10-01 2023-03-01 _huc03 _90-100
done 6 2022-10-01 2023-03-01 _huc03 _90-100
done 7 2022-10-01 2023-03-01 _huc03 _90-100
done 8 2022-10-01 2023-03-01 _huc03 _90-100
done 9 2022-10-01 2023-03-01 _huc03 _90-100


In [16]:
###HUC03
#129 Sites
#4/18/23


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc03_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(80,90)
huc = '_huc03'
siteSpan = '_80-90'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc03 _80-90
done 1 2013-03-18 2013-11-01 _huc03 _80-90
done 2 2013-03-18 2013-11-01 _huc03 _80-90
done 3 2013-03-18 2013-11-01 _huc03 _80-90
done 4 2013-03-18 2013-11-01 _huc03 _80-90
done 5 2013-03-18 2013-11-01 _huc03 _80-90
done 6 2013-03-18 2013-11-01 _huc03 _80-90
done 7 2013-03-18 2013-11-01 _huc03 _80-90
done 8 2013-03-18 2013-11-01 _huc03 _80-90
done 9 2013-03-18 2013-11-01 _huc03 _80-90
done 0 2013-10-01 2014-05-01 _huc03 _80-90
done 1 2013-10-01 2014-05-01 _huc03 _80-90
done 2 2013-10-01 2014-05-01 _huc03 _80-90
done 3 2013-10-01 2014-05-01 _huc03 _80-90
done 4 2013-10-01 2014-05-01 _huc03 _80-90
done 5 2013-10-01 2014-05-01 _huc03 _80-90
done 6 2013-10-01 2014-05-01 _huc03 _80-90
done 7 2013-10-01 2014-05-01 _huc03 _80-90
done 8 2013-10-01 2014-05-01 _huc03 _80-90
done 9 2013-10-01 2014-05-01 _huc03 _80-90
done 0 2014-04-01 2014-11-01 _huc03 _80-90
done 1 2014-04-01 2014-11-01 _huc03 _80-90
done 2 2014-04-01 2014-11-01 _huc03 _80-90
done 3 2014

done 1 2022-10-01 2023-03-01 _huc03 _80-90
done 2 2022-10-01 2023-03-01 _huc03 _80-90
done 3 2022-10-01 2023-03-01 _huc03 _80-90
done 4 2022-10-01 2023-03-01 _huc03 _80-90
done 5 2022-10-01 2023-03-01 _huc03 _80-90
done 6 2022-10-01 2023-03-01 _huc03 _80-90
done 7 2022-10-01 2023-03-01 _huc03 _80-90
done 8 2022-10-01 2023-03-01 _huc03 _80-90
done 9 2022-10-01 2023-03-01 _huc03 _80-90


In [14]:
###HUC03
#129 Sites
#4/18/23


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc03_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(70,80)
huc = '_huc03'
siteSpan = '_70-80'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc03 _70-80
done 1 2013-03-18 2013-11-01 _huc03 _70-80
done 2 2013-03-18 2013-11-01 _huc03 _70-80
done 3 2013-03-18 2013-11-01 _huc03 _70-80
done 4 2013-03-18 2013-11-01 _huc03 _70-80
done 5 2013-03-18 2013-11-01 _huc03 _70-80
done 6 2013-03-18 2013-11-01 _huc03 _70-80
done 7 2013-03-18 2013-11-01 _huc03 _70-80
done 8 2013-03-18 2013-11-01 _huc03 _70-80
done 9 2013-03-18 2013-11-01 _huc03 _70-80
done 0 2013-10-01 2014-05-01 _huc03 _70-80
done 1 2013-10-01 2014-05-01 _huc03 _70-80
done 2 2013-10-01 2014-05-01 _huc03 _70-80
done 3 2013-10-01 2014-05-01 _huc03 _70-80
done 4 2013-10-01 2014-05-01 _huc03 _70-80
done 5 2013-10-01 2014-05-01 _huc03 _70-80
done 6 2013-10-01 2014-05-01 _huc03 _70-80
done 7 2013-10-01 2014-05-01 _huc03 _70-80
done 8 2013-10-01 2014-05-01 _huc03 _70-80
done 9 2013-10-01 2014-05-01 _huc03 _70-80
done 0 2014-04-01 2014-11-01 _huc03 _70-80
done 1 2014-04-01 2014-11-01 _huc03 _70-80
done 2 2014-04-01 2014-11-01 _huc03 _70-80
done 3 2014

done 1 2022-10-01 2023-03-01 _huc03 _70-80
done 2 2022-10-01 2023-03-01 _huc03 _70-80
done 3 2022-10-01 2023-03-01 _huc03 _70-80
done 4 2022-10-01 2023-03-01 _huc03 _70-80
done 5 2022-10-01 2023-03-01 _huc03 _70-80
done 6 2022-10-01 2023-03-01 _huc03 _70-80
done 7 2022-10-01 2023-03-01 _huc03 _70-80
done 8 2022-10-01 2023-03-01 _huc03 _70-80
done 9 2022-10-01 2023-03-01 _huc03 _70-80


In [12]:
###HUC03
#129 Sites
#4/18/23


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc03_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(60,70)
huc = '_huc03'
siteSpan = '_60-70'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc03 _60-70
done 1 2013-03-18 2013-11-01 _huc03 _60-70
done 2 2013-03-18 2013-11-01 _huc03 _60-70
done 3 2013-03-18 2013-11-01 _huc03 _60-70
done 4 2013-03-18 2013-11-01 _huc03 _60-70
done 5 2013-03-18 2013-11-01 _huc03 _60-70
done 6 2013-03-18 2013-11-01 _huc03 _60-70
done 7 2013-03-18 2013-11-01 _huc03 _60-70
done 8 2013-03-18 2013-11-01 _huc03 _60-70
done 9 2013-03-18 2013-11-01 _huc03 _60-70
done 0 2013-10-01 2014-05-01 _huc03 _60-70
done 1 2013-10-01 2014-05-01 _huc03 _60-70
done 2 2013-10-01 2014-05-01 _huc03 _60-70
done 3 2013-10-01 2014-05-01 _huc03 _60-70
done 4 2013-10-01 2014-05-01 _huc03 _60-70
done 5 2013-10-01 2014-05-01 _huc03 _60-70
done 6 2013-10-01 2014-05-01 _huc03 _60-70
done 7 2013-10-01 2014-05-01 _huc03 _60-70
done 8 2013-10-01 2014-05-01 _huc03 _60-70
done 9 2013-10-01 2014-05-01 _huc03 _60-70
done 0 2014-04-01 2014-11-01 _huc03 _60-70
done 1 2014-04-01 2014-11-01 _huc03 _60-70
done 2 2014-04-01 2014-11-01 _huc03 _60-70
done 3 2014

done 1 2022-10-01 2023-03-01 _huc03 _60-70
done 2 2022-10-01 2023-03-01 _huc03 _60-70
done 3 2022-10-01 2023-03-01 _huc03 _60-70
done 4 2022-10-01 2023-03-01 _huc03 _60-70
done 5 2022-10-01 2023-03-01 _huc03 _60-70
done 6 2022-10-01 2023-03-01 _huc03 _60-70
done 7 2022-10-01 2023-03-01 _huc03 _60-70
done 8 2022-10-01 2023-03-01 _huc03 _60-70
done 9 2022-10-01 2023-03-01 _huc03 _60-70


In [10]:
###HUC03
#129 Sites
#4/18/23


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc03_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(50,60)
huc = '_huc03'
siteSpan = '_50-60'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc03 _50-60
done 1 2013-03-18 2013-11-01 _huc03 _50-60
done 2 2013-03-18 2013-11-01 _huc03 _50-60
done 3 2013-03-18 2013-11-01 _huc03 _50-60
done 4 2013-03-18 2013-11-01 _huc03 _50-60
done 5 2013-03-18 2013-11-01 _huc03 _50-60
done 6 2013-03-18 2013-11-01 _huc03 _50-60
done 7 2013-03-18 2013-11-01 _huc03 _50-60
done 8 2013-03-18 2013-11-01 _huc03 _50-60
done 9 2013-03-18 2013-11-01 _huc03 _50-60
done 0 2013-10-01 2014-05-01 _huc03 _50-60
done 1 2013-10-01 2014-05-01 _huc03 _50-60
done 2 2013-10-01 2014-05-01 _huc03 _50-60
done 3 2013-10-01 2014-05-01 _huc03 _50-60
done 4 2013-10-01 2014-05-01 _huc03 _50-60
done 5 2013-10-01 2014-05-01 _huc03 _50-60
done 6 2013-10-01 2014-05-01 _huc03 _50-60
done 7 2013-10-01 2014-05-01 _huc03 _50-60
done 8 2013-10-01 2014-05-01 _huc03 _50-60
done 9 2013-10-01 2014-05-01 _huc03 _50-60
done 0 2014-04-01 2014-11-01 _huc03 _50-60
done 1 2014-04-01 2014-11-01 _huc03 _50-60
done 2 2014-04-01 2014-11-01 _huc03 _50-60
done 3 2014

done 1 2022-10-01 2023-03-01 _huc03 _50-60
done 2 2022-10-01 2023-03-01 _huc03 _50-60
done 3 2022-10-01 2023-03-01 _huc03 _50-60
done 4 2022-10-01 2023-03-01 _huc03 _50-60
done 5 2022-10-01 2023-03-01 _huc03 _50-60
done 6 2022-10-01 2023-03-01 _huc03 _50-60
done 7 2022-10-01 2023-03-01 _huc03 _50-60
done 8 2022-10-01 2023-03-01 _huc03 _50-60
done 9 2022-10-01 2023-03-01 _huc03 _50-60


In [9]:
###HUC03
#129 Sites
#4/18/23


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc03_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(40,50)
huc = '_huc03'
siteSpan = '_40-50' 

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc03 _40-50
done 1 2013-03-18 2013-11-01 _huc03 _40-50
done 2 2013-03-18 2013-11-01 _huc03 _40-50
done 3 2013-03-18 2013-11-01 _huc03 _40-50
done 4 2013-03-18 2013-11-01 _huc03 _40-50
done 5 2013-03-18 2013-11-01 _huc03 _40-50
done 6 2013-03-18 2013-11-01 _huc03 _40-50
done 7 2013-03-18 2013-11-01 _huc03 _40-50
done 8 2013-03-18 2013-11-01 _huc03 _40-50
done 9 2013-03-18 2013-11-01 _huc03 _40-50
done 0 2013-10-01 2014-05-01 _huc03 _40-50
done 1 2013-10-01 2014-05-01 _huc03 _40-50
done 2 2013-10-01 2014-05-01 _huc03 _40-50
done 3 2013-10-01 2014-05-01 _huc03 _40-50
done 4 2013-10-01 2014-05-01 _huc03 _40-50
done 5 2013-10-01 2014-05-01 _huc03 _40-50
done 6 2013-10-01 2014-05-01 _huc03 _40-50
done 7 2013-10-01 2014-05-01 _huc03 _40-50
done 8 2013-10-01 2014-05-01 _huc03 _40-50
done 9 2013-10-01 2014-05-01 _huc03 _40-50
done 0 2014-04-01 2014-11-01 _huc03 _40-50
done 1 2014-04-01 2014-11-01 _huc03 _40-50
done 2 2014-04-01 2014-11-01 _huc03 _40-50
done 3 2014

done 1 2022-10-01 2023-03-01 _huc03 _40-50
done 2 2022-10-01 2023-03-01 _huc03 _40-50
done 3 2022-10-01 2023-03-01 _huc03 _40-50
done 4 2022-10-01 2023-03-01 _huc03 _40-50
done 5 2022-10-01 2023-03-01 _huc03 _40-50
done 6 2022-10-01 2023-03-01 _huc03 _40-50
done 7 2022-10-01 2023-03-01 _huc03 _40-50
done 8 2022-10-01 2023-03-01 _huc03 _40-50
done 9 2022-10-01 2023-03-01 _huc03 _40-50


In [7]:
###HUC03
#129 Sites
#4/18/23


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc03_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(30,40)
huc = '_huc03'
siteSpan = '_30-40' 

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc03 _30-40
done 1 2013-03-18 2013-11-01 _huc03 _30-40
done 2 2013-03-18 2013-11-01 _huc03 _30-40
done 3 2013-03-18 2013-11-01 _huc03 _30-40
done 4 2013-03-18 2013-11-01 _huc03 _30-40
done 5 2013-03-18 2013-11-01 _huc03 _30-40
done 6 2013-03-18 2013-11-01 _huc03 _30-40
done 7 2013-03-18 2013-11-01 _huc03 _30-40
done 8 2013-03-18 2013-11-01 _huc03 _30-40
done 9 2013-03-18 2013-11-01 _huc03 _30-40
done 0 2013-10-01 2014-05-01 _huc03 _30-40
done 1 2013-10-01 2014-05-01 _huc03 _30-40
done 2 2013-10-01 2014-05-01 _huc03 _30-40
done 3 2013-10-01 2014-05-01 _huc03 _30-40
done 4 2013-10-01 2014-05-01 _huc03 _30-40
done 5 2013-10-01 2014-05-01 _huc03 _30-40
done 6 2013-10-01 2014-05-01 _huc03 _30-40
done 7 2013-10-01 2014-05-01 _huc03 _30-40
done 8 2013-10-01 2014-05-01 _huc03 _30-40
done 9 2013-10-01 2014-05-01 _huc03 _30-40
done 0 2014-04-01 2014-11-01 _huc03 _30-40
done 1 2014-04-01 2014-11-01 _huc03 _30-40
done 2 2014-04-01 2014-11-01 _huc03 _30-40
done 3 2014

done 1 2022-10-01 2023-03-01 _huc03 _30-40
done 2 2022-10-01 2023-03-01 _huc03 _30-40
done 3 2022-10-01 2023-03-01 _huc03 _30-40
done 4 2022-10-01 2023-03-01 _huc03 _30-40
done 5 2022-10-01 2023-03-01 _huc03 _30-40
done 6 2022-10-01 2023-03-01 _huc03 _30-40
done 7 2022-10-01 2023-03-01 _huc03 _30-40
done 8 2022-10-01 2023-03-01 _huc03 _30-40
done 9 2022-10-01 2023-03-01 _huc03 _30-40


In [2]:
###HUC03
#129 Sites
#4/14/23

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc03_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(0,10)
huc = '_huc03'
siteSpan = '_0-10' 

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc03 _0-10
done 1 2013-03-18 2013-11-01 _huc03 _0-10
done 2 2013-03-18 2013-11-01 _huc03 _0-10
done 3 2013-03-18 2013-11-01 _huc03 _0-10
done 4 2013-03-18 2013-11-01 _huc03 _0-10
done 5 2013-03-18 2013-11-01 _huc03 _0-10
done 6 2013-03-18 2013-11-01 _huc03 _0-10
done 7 2013-03-18 2013-11-01 _huc03 _0-10
done 8 2013-03-18 2013-11-01 _huc03 _0-10
done 9 2013-03-18 2013-11-01 _huc03 _0-10
done 0 2013-10-01 2014-05-01 _huc03 _0-10
done 1 2013-10-01 2014-05-01 _huc03 _0-10
done 2 2013-10-01 2014-05-01 _huc03 _0-10
done 3 2013-10-01 2014-05-01 _huc03 _0-10
done 4 2013-10-01 2014-05-01 _huc03 _0-10
done 5 2013-10-01 2014-05-01 _huc03 _0-10
done 6 2013-10-01 2014-05-01 _huc03 _0-10
done 7 2013-10-01 2014-05-01 _huc03 _0-10
done 8 2013-10-01 2014-05-01 _huc03 _0-10
done 9 2013-10-01 2014-05-01 _huc03 _0-10
done 0 2014-04-01 2014-11-01 _huc03 _0-10
done 1 2014-04-01 2014-11-01 _huc03 _0-10
done 2 2014-04-01 2014-11-01 _huc03 _0-10
done 3 2014-04-01 2014-11-01 _huc0

done 6 2022-10-01 2023-03-01 _huc03 _0-10
done 7 2022-10-01 2023-03-01 _huc03 _0-10
done 8 2022-10-01 2023-03-01 _huc03 _0-10
done 9 2022-10-01 2023-03-01 _huc03 _0-10


In [2]:
###HUC03
#129 Sites
#4/18/23

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc03_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(10,20)
huc = '_huc03'
siteSpan = '_10-20' 

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc03 _10-20
done 1 2013-03-18 2013-11-01 _huc03 _10-20
done 2 2013-03-18 2013-11-01 _huc03 _10-20
done 3 2013-03-18 2013-11-01 _huc03 _10-20
done 4 2013-03-18 2013-11-01 _huc03 _10-20
done 5 2013-03-18 2013-11-01 _huc03 _10-20
done 6 2013-03-18 2013-11-01 _huc03 _10-20
done 7 2013-03-18 2013-11-01 _huc03 _10-20
done 8 2013-03-18 2013-11-01 _huc03 _10-20
done 9 2013-03-18 2013-11-01 _huc03 _10-20
done 0 2013-10-01 2014-05-01 _huc03 _10-20
done 1 2013-10-01 2014-05-01 _huc03 _10-20
done 2 2013-10-01 2014-05-01 _huc03 _10-20
done 3 2013-10-01 2014-05-01 _huc03 _10-20
done 4 2013-10-01 2014-05-01 _huc03 _10-20
done 5 2013-10-01 2014-05-01 _huc03 _10-20
done 6 2013-10-01 2014-05-01 _huc03 _10-20
done 7 2013-10-01 2014-05-01 _huc03 _10-20
done 8 2013-10-01 2014-05-01 _huc03 _10-20
done 9 2013-10-01 2014-05-01 _huc03 _10-20
done 0 2014-04-01 2014-11-01 _huc03 _10-20
done 1 2014-04-01 2014-11-01 _huc03 _10-20
done 2 2014-04-01 2014-11-01 _huc03 _10-20
done 3 2014

done 1 2022-10-01 2023-03-01 _huc03 _10-20
done 2 2022-10-01 2023-03-01 _huc03 _10-20
done 3 2022-10-01 2023-03-01 _huc03 _10-20
done 4 2022-10-01 2023-03-01 _huc03 _10-20
done 5 2022-10-01 2023-03-01 _huc03 _10-20
done 6 2022-10-01 2023-03-01 _huc03 _10-20
done 7 2022-10-01 2023-03-01 _huc03 _10-20
done 8 2022-10-01 2023-03-01 _huc03 _10-20
done 9 2022-10-01 2023-03-01 _huc03 _10-20


In [5]:
###HUC03
#129 Sites
#4/18/23

datesStag = (('2019-04-01', '2019-11-01'),
        ('2019-10-01', '2020-05-01'),
        ('2020-04-01', '2020-11-01'),
        ('2020-10-01', '2021-05-01'),
        ('2021-04-01', '2021-11-01'),
        ('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc03_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(20,30)
huc = '_huc03'
siteSpan = '_20-30' 

#Call function
featTest(hucList)

done 0 2019-04-01 2019-11-01 _huc03 _20-30
done 1 2019-04-01 2019-11-01 _huc03 _20-30
done 2 2019-04-01 2019-11-01 _huc03 _20-30
done 3 2019-04-01 2019-11-01 _huc03 _20-30
done 4 2019-04-01 2019-11-01 _huc03 _20-30
done 5 2019-04-01 2019-11-01 _huc03 _20-30
done 6 2019-04-01 2019-11-01 _huc03 _20-30
done 7 2019-04-01 2019-11-01 _huc03 _20-30
done 8 2019-04-01 2019-11-01 _huc03 _20-30
done 9 2019-04-01 2019-11-01 _huc03 _20-30
done 0 2019-10-01 2020-05-01 _huc03 _20-30
done 1 2019-10-01 2020-05-01 _huc03 _20-30
done 2 2019-10-01 2020-05-01 _huc03 _20-30
done 3 2019-10-01 2020-05-01 _huc03 _20-30
done 4 2019-10-01 2020-05-01 _huc03 _20-30
done 5 2019-10-01 2020-05-01 _huc03 _20-30
done 6 2019-10-01 2020-05-01 _huc03 _20-30
done 7 2019-10-01 2020-05-01 _huc03 _20-30
done 8 2019-10-01 2020-05-01 _huc03 _20-30
done 9 2019-10-01 2020-05-01 _huc03 _20-30
done 0 2020-04-01 2020-11-01 _huc03 _20-30
done 1 2020-04-01 2020-11-01 _huc03 _20-30
done 2 2020-04-01 2020-11-01 _huc03 _20-30
done 3 2020

In [19]:
###HUC17
#98 Sites
#4/9/23

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc17_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(0,10)
huc = '_huc17'
siteSpan = '_0-10' 

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc17 _0-10
done 1 2013-03-18 2013-11-01 _huc17 _0-10
done 2 2013-03-18 2013-11-01 _huc17 _0-10
done 3 2013-03-18 2013-11-01 _huc17 _0-10
done 4 2013-03-18 2013-11-01 _huc17 _0-10
done 5 2013-03-18 2013-11-01 _huc17 _0-10
done 6 2013-03-18 2013-11-01 _huc17 _0-10
done 7 2013-03-18 2013-11-01 _huc17 _0-10
done 8 2013-03-18 2013-11-01 _huc17 _0-10
done 9 2013-03-18 2013-11-01 _huc17 _0-10
done 0 2013-10-01 2014-05-01 _huc17 _0-10
done 1 2013-10-01 2014-05-01 _huc17 _0-10
done 2 2013-10-01 2014-05-01 _huc17 _0-10
done 3 2013-10-01 2014-05-01 _huc17 _0-10
done 4 2013-10-01 2014-05-01 _huc17 _0-10
done 5 2013-10-01 2014-05-01 _huc17 _0-10
done 6 2013-10-01 2014-05-01 _huc17 _0-10
done 7 2013-10-01 2014-05-01 _huc17 _0-10
done 8 2013-10-01 2014-05-01 _huc17 _0-10
done 9 2013-10-01 2014-05-01 _huc17 _0-10
done 0 2014-04-01 2014-11-01 _huc17 _0-10
done 1 2014-04-01 2014-11-01 _huc17 _0-10
done 2 2014-04-01 2014-11-01 _huc17 _0-10
done 3 2014-04-01 2014-11-01 _huc1

done 6 2022-10-01 2023-03-01 _huc17 _0-10
done 7 2022-10-01 2023-03-01 _huc17 _0-10
done 8 2022-10-01 2023-03-01 _huc17 _0-10
done 9 2022-10-01 2023-03-01 _huc17 _0-10


In [21]:
###HUC17
#98 Sites
#4/9/23

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc17_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(10,20)
huc = '_huc17'
siteSpan = '_10-20' 

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc17 _10-20
done 1 2013-03-18 2013-11-01 _huc17 _10-20
done 2 2013-03-18 2013-11-01 _huc17 _10-20
done 3 2013-03-18 2013-11-01 _huc17 _10-20
done 4 2013-03-18 2013-11-01 _huc17 _10-20
done 5 2013-03-18 2013-11-01 _huc17 _10-20
done 6 2013-03-18 2013-11-01 _huc17 _10-20
done 7 2013-03-18 2013-11-01 _huc17 _10-20
done 8 2013-03-18 2013-11-01 _huc17 _10-20
done 9 2013-03-18 2013-11-01 _huc17 _10-20
done 0 2013-10-01 2014-05-01 _huc17 _10-20
done 1 2013-10-01 2014-05-01 _huc17 _10-20
done 2 2013-10-01 2014-05-01 _huc17 _10-20
done 3 2013-10-01 2014-05-01 _huc17 _10-20
done 4 2013-10-01 2014-05-01 _huc17 _10-20
done 5 2013-10-01 2014-05-01 _huc17 _10-20
done 6 2013-10-01 2014-05-01 _huc17 _10-20
done 7 2013-10-01 2014-05-01 _huc17 _10-20
done 8 2013-10-01 2014-05-01 _huc17 _10-20
done 9 2013-10-01 2014-05-01 _huc17 _10-20
done 0 2014-04-01 2014-11-01 _huc17 _10-20
done 1 2014-04-01 2014-11-01 _huc17 _10-20
done 2 2014-04-01 2014-11-01 _huc17 _10-20
done 3 2014

done 1 2022-10-01 2023-03-01 _huc17 _10-20
done 2 2022-10-01 2023-03-01 _huc17 _10-20
done 3 2022-10-01 2023-03-01 _huc17 _10-20
done 4 2022-10-01 2023-03-01 _huc17 _10-20
done 5 2022-10-01 2023-03-01 _huc17 _10-20
done 6 2022-10-01 2023-03-01 _huc17 _10-20
done 7 2022-10-01 2023-03-01 _huc17 _10-20
done 8 2022-10-01 2023-03-01 _huc17 _10-20
done 9 2022-10-01 2023-03-01 _huc17 _10-20


In [23]:
###HUC17
#98 Sites
#4/9/23

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc17_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(20,30)
huc = '_huc17'
siteSpan = '_20-30' 

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc17 _20-30
done 1 2013-03-18 2013-11-01 _huc17 _20-30
done 2 2013-03-18 2013-11-01 _huc17 _20-30
done 3 2013-03-18 2013-11-01 _huc17 _20-30
done 4 2013-03-18 2013-11-01 _huc17 _20-30
done 5 2013-03-18 2013-11-01 _huc17 _20-30
done 6 2013-03-18 2013-11-01 _huc17 _20-30
done 7 2013-03-18 2013-11-01 _huc17 _20-30
done 8 2013-03-18 2013-11-01 _huc17 _20-30
done 9 2013-03-18 2013-11-01 _huc17 _20-30
done 0 2013-10-01 2014-05-01 _huc17 _20-30
done 1 2013-10-01 2014-05-01 _huc17 _20-30
done 2 2013-10-01 2014-05-01 _huc17 _20-30
done 3 2013-10-01 2014-05-01 _huc17 _20-30
done 4 2013-10-01 2014-05-01 _huc17 _20-30
done 5 2013-10-01 2014-05-01 _huc17 _20-30
done 6 2013-10-01 2014-05-01 _huc17 _20-30
done 7 2013-10-01 2014-05-01 _huc17 _20-30
done 8 2013-10-01 2014-05-01 _huc17 _20-30
done 9 2013-10-01 2014-05-01 _huc17 _20-30
done 0 2014-04-01 2014-11-01 _huc17 _20-30
done 1 2014-04-01 2014-11-01 _huc17 _20-30
done 2 2014-04-01 2014-11-01 _huc17 _20-30
done 3 2014

done 1 2022-10-01 2023-03-01 _huc17 _20-30
done 2 2022-10-01 2023-03-01 _huc17 _20-30
done 3 2022-10-01 2023-03-01 _huc17 _20-30
done 4 2022-10-01 2023-03-01 _huc17 _20-30
done 5 2022-10-01 2023-03-01 _huc17 _20-30
done 6 2022-10-01 2023-03-01 _huc17 _20-30
done 7 2022-10-01 2023-03-01 _huc17 _20-30
done 8 2022-10-01 2023-03-01 _huc17 _20-30
done 9 2022-10-01 2023-03-01 _huc17 _20-30


In [25]:
###HUC17
#98 Sites
#4/9/23

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc17_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(30,40)
huc = '_huc17'
siteSpan = '_30-40' 

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc17 _30-40
done 1 2013-03-18 2013-11-01 _huc17 _30-40
done 2 2013-03-18 2013-11-01 _huc17 _30-40
done 3 2013-03-18 2013-11-01 _huc17 _30-40
done 4 2013-03-18 2013-11-01 _huc17 _30-40
done 5 2013-03-18 2013-11-01 _huc17 _30-40
done 6 2013-03-18 2013-11-01 _huc17 _30-40
done 7 2013-03-18 2013-11-01 _huc17 _30-40
done 8 2013-03-18 2013-11-01 _huc17 _30-40
done 9 2013-03-18 2013-11-01 _huc17 _30-40
done 0 2013-10-01 2014-05-01 _huc17 _30-40
done 1 2013-10-01 2014-05-01 _huc17 _30-40
done 2 2013-10-01 2014-05-01 _huc17 _30-40
done 3 2013-10-01 2014-05-01 _huc17 _30-40
done 4 2013-10-01 2014-05-01 _huc17 _30-40
done 5 2013-10-01 2014-05-01 _huc17 _30-40
done 6 2013-10-01 2014-05-01 _huc17 _30-40
done 7 2013-10-01 2014-05-01 _huc17 _30-40
done 8 2013-10-01 2014-05-01 _huc17 _30-40
done 9 2013-10-01 2014-05-01 _huc17 _30-40
done 0 2014-04-01 2014-11-01 _huc17 _30-40
done 1 2014-04-01 2014-11-01 _huc17 _30-40
done 2 2014-04-01 2014-11-01 _huc17 _30-40
done 3 2014

done 1 2022-10-01 2023-03-01 _huc17 _30-40
done 2 2022-10-01 2023-03-01 _huc17 _30-40
done 3 2022-10-01 2023-03-01 _huc17 _30-40
done 4 2022-10-01 2023-03-01 _huc17 _30-40
done 5 2022-10-01 2023-03-01 _huc17 _30-40
done 6 2022-10-01 2023-03-01 _huc17 _30-40
done 7 2022-10-01 2023-03-01 _huc17 _30-40
done 8 2022-10-01 2023-03-01 _huc17 _30-40
done 9 2022-10-01 2023-03-01 _huc17 _30-40


In [27]:
###HUC17
#98 Sites
#4/9/23

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc17_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(40,50)
huc = '_huc17'
siteSpan = '_40-50' 

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc17 _40-50
done 1 2013-03-18 2013-11-01 _huc17 _40-50
done 2 2013-03-18 2013-11-01 _huc17 _40-50
done 3 2013-03-18 2013-11-01 _huc17 _40-50
done 4 2013-03-18 2013-11-01 _huc17 _40-50
done 5 2013-03-18 2013-11-01 _huc17 _40-50
done 6 2013-03-18 2013-11-01 _huc17 _40-50
done 7 2013-03-18 2013-11-01 _huc17 _40-50
done 8 2013-03-18 2013-11-01 _huc17 _40-50
done 9 2013-03-18 2013-11-01 _huc17 _40-50
done 0 2013-10-01 2014-05-01 _huc17 _40-50
done 1 2013-10-01 2014-05-01 _huc17 _40-50
done 2 2013-10-01 2014-05-01 _huc17 _40-50
done 3 2013-10-01 2014-05-01 _huc17 _40-50
done 4 2013-10-01 2014-05-01 _huc17 _40-50
done 5 2013-10-01 2014-05-01 _huc17 _40-50
done 6 2013-10-01 2014-05-01 _huc17 _40-50
done 7 2013-10-01 2014-05-01 _huc17 _40-50
done 8 2013-10-01 2014-05-01 _huc17 _40-50
done 9 2013-10-01 2014-05-01 _huc17 _40-50
done 0 2014-04-01 2014-11-01 _huc17 _40-50
done 1 2014-04-01 2014-11-01 _huc17 _40-50
done 2 2014-04-01 2014-11-01 _huc17 _40-50
done 3 2014

done 1 2022-10-01 2023-03-01 _huc17 _40-50
done 2 2022-10-01 2023-03-01 _huc17 _40-50
done 3 2022-10-01 2023-03-01 _huc17 _40-50
done 4 2022-10-01 2023-03-01 _huc17 _40-50
done 5 2022-10-01 2023-03-01 _huc17 _40-50
done 6 2022-10-01 2023-03-01 _huc17 _40-50
done 7 2022-10-01 2023-03-01 _huc17 _40-50
done 8 2022-10-01 2023-03-01 _huc17 _40-50
done 9 2022-10-01 2023-03-01 _huc17 _40-50


In [29]:
###HUC17
#98 Sites
#4/9/23

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc17_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(50,60)
huc = '_huc17'
siteSpan = '_50-60' 

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc17 _50-60
done 1 2013-03-18 2013-11-01 _huc17 _50-60
done 2 2013-03-18 2013-11-01 _huc17 _50-60
done 3 2013-03-18 2013-11-01 _huc17 _50-60
done 4 2013-03-18 2013-11-01 _huc17 _50-60
done 5 2013-03-18 2013-11-01 _huc17 _50-60
done 6 2013-03-18 2013-11-01 _huc17 _50-60
done 7 2013-03-18 2013-11-01 _huc17 _50-60
done 8 2013-03-18 2013-11-01 _huc17 _50-60
done 9 2013-03-18 2013-11-01 _huc17 _50-60
done 0 2013-10-01 2014-05-01 _huc17 _50-60
done 1 2013-10-01 2014-05-01 _huc17 _50-60
done 2 2013-10-01 2014-05-01 _huc17 _50-60
done 3 2013-10-01 2014-05-01 _huc17 _50-60
done 4 2013-10-01 2014-05-01 _huc17 _50-60
done 5 2013-10-01 2014-05-01 _huc17 _50-60
done 6 2013-10-01 2014-05-01 _huc17 _50-60
done 7 2013-10-01 2014-05-01 _huc17 _50-60
done 8 2013-10-01 2014-05-01 _huc17 _50-60
done 9 2013-10-01 2014-05-01 _huc17 _50-60
done 0 2014-04-01 2014-11-01 _huc17 _50-60
done 1 2014-04-01 2014-11-01 _huc17 _50-60
done 2 2014-04-01 2014-11-01 _huc17 _50-60
done 3 2014

done 1 2022-10-01 2023-03-01 _huc17 _50-60
done 2 2022-10-01 2023-03-01 _huc17 _50-60
done 3 2022-10-01 2023-03-01 _huc17 _50-60
done 4 2022-10-01 2023-03-01 _huc17 _50-60
done 5 2022-10-01 2023-03-01 _huc17 _50-60
done 6 2022-10-01 2023-03-01 _huc17 _50-60
done 7 2022-10-01 2023-03-01 _huc17 _50-60
done 8 2022-10-01 2023-03-01 _huc17 _50-60
done 9 2022-10-01 2023-03-01 _huc17 _50-60


In [33]:
###HUC17
#98 Sites
#4/9/23

datesStag = (('2014-04-01', '2014-11-01'),
        ('2014-10-01', '2015-05-01'),
        ('2015-04-01', '2015-11-01'),
        ('2015-10-01', '2016-05-01'),
        ('2016-04-01', '2016-11-01'),
        ('2016-10-01', '2017-05-01'),
        ('2017-04-01', '2017-11-01'),
        ('2017-10-01', '2018-05-01'),
        ('2018-04-01', '2018-11-01'),
        ('2018-10-01', '2019-05-01'),
        ('2019-04-01', '2019-11-01'),
        ('2019-10-01', '2020-05-01'),
        ('2020-04-01', '2020-11-01'),
        ('2020-10-01', '2021-05-01'),
        ('2021-04-01', '2021-11-01'),
        ('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc17_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(60,70)
huc = '_huc17'
siteSpan = '_60-70' 

#Call function
featTest(hucList)

done 0 2014-04-01 2014-11-01 _huc17 _60-70
done 1 2014-04-01 2014-11-01 _huc17 _60-70
done 2 2014-04-01 2014-11-01 _huc17 _60-70
done 3 2014-04-01 2014-11-01 _huc17 _60-70
done 4 2014-04-01 2014-11-01 _huc17 _60-70
done 5 2014-04-01 2014-11-01 _huc17 _60-70
done 6 2014-04-01 2014-11-01 _huc17 _60-70
done 7 2014-04-01 2014-11-01 _huc17 _60-70
done 8 2014-04-01 2014-11-01 _huc17 _60-70
done 9 2014-04-01 2014-11-01 _huc17 _60-70
done 0 2014-10-01 2015-05-01 _huc17 _60-70
done 1 2014-10-01 2015-05-01 _huc17 _60-70
done 2 2014-10-01 2015-05-01 _huc17 _60-70
done 3 2014-10-01 2015-05-01 _huc17 _60-70
done 4 2014-10-01 2015-05-01 _huc17 _60-70
done 5 2014-10-01 2015-05-01 _huc17 _60-70
done 6 2014-10-01 2015-05-01 _huc17 _60-70
done 7 2014-10-01 2015-05-01 _huc17 _60-70
done 8 2014-10-01 2015-05-01 _huc17 _60-70
done 9 2014-10-01 2015-05-01 _huc17 _60-70
done 0 2015-04-01 2015-11-01 _huc17 _60-70
done 1 2015-04-01 2015-11-01 _huc17 _60-70
done 2 2015-04-01 2015-11-01 _huc17 _60-70
done 3 2015

In [35]:
###HUC17
#98 Sites
#4/9/23

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc17_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(70,80)
huc = '_huc17'
siteSpan = '_70-80' 

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc17 _70-80
done 1 2013-03-18 2013-11-01 _huc17 _70-80
done 2 2013-03-18 2013-11-01 _huc17 _70-80
done 3 2013-03-18 2013-11-01 _huc17 _70-80
done 4 2013-03-18 2013-11-01 _huc17 _70-80
done 5 2013-03-18 2013-11-01 _huc17 _70-80
done 6 2013-03-18 2013-11-01 _huc17 _70-80
done 7 2013-03-18 2013-11-01 _huc17 _70-80
done 8 2013-03-18 2013-11-01 _huc17 _70-80
done 9 2013-03-18 2013-11-01 _huc17 _70-80
done 0 2013-10-01 2014-05-01 _huc17 _70-80
done 1 2013-10-01 2014-05-01 _huc17 _70-80
done 2 2013-10-01 2014-05-01 _huc17 _70-80
done 3 2013-10-01 2014-05-01 _huc17 _70-80
done 4 2013-10-01 2014-05-01 _huc17 _70-80
done 5 2013-10-01 2014-05-01 _huc17 _70-80
done 6 2013-10-01 2014-05-01 _huc17 _70-80
done 7 2013-10-01 2014-05-01 _huc17 _70-80
done 8 2013-10-01 2014-05-01 _huc17 _70-80
done 9 2013-10-01 2014-05-01 _huc17 _70-80
done 0 2014-04-01 2014-11-01 _huc17 _70-80
done 1 2014-04-01 2014-11-01 _huc17 _70-80
done 2 2014-04-01 2014-11-01 _huc17 _70-80
done 3 2014

done 1 2022-10-01 2023-03-01 _huc17 _70-80
done 2 2022-10-01 2023-03-01 _huc17 _70-80
done 3 2022-10-01 2023-03-01 _huc17 _70-80
done 4 2022-10-01 2023-03-01 _huc17 _70-80
done 5 2022-10-01 2023-03-01 _huc17 _70-80
done 6 2022-10-01 2023-03-01 _huc17 _70-80
done 7 2022-10-01 2023-03-01 _huc17 _70-80
done 8 2022-10-01 2023-03-01 _huc17 _70-80
done 9 2022-10-01 2023-03-01 _huc17 _70-80


In [42]:
###HUC17
#98 Sites
#4/9/23

datesStag = (('2022-10-01', '2023-03-01'),('2022-04-01', '2022-11-01'))
#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc17_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(80,90)
huc = '_huc17'
siteSpan = '_80-90' 

#Call function
featTest(hucList)

done 0 2022-10-01 2023-03-01 _huc17 _80-90
done 1 2022-10-01 2023-03-01 _huc17 _80-90
done 2 2022-10-01 2023-03-01 _huc17 _80-90
done 3 2022-10-01 2023-03-01 _huc17 _80-90
done 4 2022-10-01 2023-03-01 _huc17 _80-90
done 5 2022-10-01 2023-03-01 _huc17 _80-90
done 6 2022-10-01 2023-03-01 _huc17 _80-90
done 7 2022-10-01 2023-03-01 _huc17 _80-90
done 8 2022-10-01 2023-03-01 _huc17 _80-90
done 9 2022-10-01 2023-03-01 _huc17 _80-90


KeyboardInterrupt: 

In [44]:
###HUC17
#98 Sites
#4/9/23

datesStag = (('2015-10-01', '2016-05-01'),
        ('2016-04-01', '2016-11-01'),
        ('2016-10-01', '2017-05-01'),
        ('2017-04-01', '2017-11-01'),
        ('2017-10-01', '2018-05-01'),
        ('2018-04-01', '2018-11-01'),
        ('2018-10-01', '2019-05-01'),
        ('2019-04-01', '2019-11-01'),
        ('2019-10-01', '2020-05-01'),
        ('2020-04-01', '2020-11-01'),
        ('2020-10-01', '2021-05-01'),
        ('2021-04-01', '2021-11-01'),
        ('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))
#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc17_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(90,98)
huc = '_huc17'
siteSpan = '_90-98' 

#Call function
featTest(hucList)

done 0 2015-10-01 2016-05-01 _huc17 _90-98
done 1 2015-10-01 2016-05-01 _huc17 _90-98
done 2 2015-10-01 2016-05-01 _huc17 _90-98
done 3 2015-10-01 2016-05-01 _huc17 _90-98
done 4 2015-10-01 2016-05-01 _huc17 _90-98
done 5 2015-10-01 2016-05-01 _huc17 _90-98
done 6 2015-10-01 2016-05-01 _huc17 _90-98
done 7 2015-10-01 2016-05-01 _huc17 _90-98
done 0 2016-04-01 2016-11-01 _huc17 _90-98
done 1 2016-04-01 2016-11-01 _huc17 _90-98
done 2 2016-04-01 2016-11-01 _huc17 _90-98
done 3 2016-04-01 2016-11-01 _huc17 _90-98
done 4 2016-04-01 2016-11-01 _huc17 _90-98
done 5 2016-04-01 2016-11-01 _huc17 _90-98
done 6 2016-04-01 2016-11-01 _huc17 _90-98
done 7 2016-04-01 2016-11-01 _huc17 _90-98
done 0 2016-10-01 2017-05-01 _huc17 _90-98
done 1 2016-10-01 2017-05-01 _huc17 _90-98
done 2 2016-10-01 2017-05-01 _huc17 _90-98
done 3 2016-10-01 2017-05-01 _huc17 _90-98
done 4 2016-10-01 2017-05-01 _huc17 _90-98
done 5 2016-10-01 2017-05-01 _huc17 _90-98
done 6 2016-10-01 2017-05-01 _huc17 _90-98
done 7 2016

In [4]:
###HUC14
#70 Sites
#4/5/23

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc14_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(0,15)
huc = '_huc14'
siteSpan = '_0-15' 

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc14 _0-15
done 1 2013-03-18 2013-11-01 _huc14 _0-15
done 2 2013-03-18 2013-11-01 _huc14 _0-15
done 3 2013-03-18 2013-11-01 _huc14 _0-15
done 4 2013-03-18 2013-11-01 _huc14 _0-15
done 5 2013-03-18 2013-11-01 _huc14 _0-15
done 6 2013-03-18 2013-11-01 _huc14 _0-15
done 7 2013-03-18 2013-11-01 _huc14 _0-15
done 8 2013-03-18 2013-11-01 _huc14 _0-15
done 9 2013-03-18 2013-11-01 _huc14 _0-15
done 10 2013-03-18 2013-11-01 _huc14 _0-15
done 11 2013-03-18 2013-11-01 _huc14 _0-15
done 12 2013-03-18 2013-11-01 _huc14 _0-15
done 13 2013-03-18 2013-11-01 _huc14 _0-15
done 14 2013-03-18 2013-11-01 _huc14 _0-15
done 0 2013-10-01 2014-05-01 _huc14 _0-15
done 1 2013-10-01 2014-05-01 _huc14 _0-15
done 2 2013-10-01 2014-05-01 _huc14 _0-15
done 3 2013-10-01 2014-05-01 _huc14 _0-15
done 4 2013-10-01 2014-05-01 _huc14 _0-15
done 5 2013-10-01 2014-05-01 _huc14 _0-15
done 6 2013-10-01 2014-05-01 _huc14 _0-15
done 7 2013-10-01 2014-05-01 _huc14 _0-15
done 8 2013-10-01 2014-05-01 

done 14 2019-04-01 2019-11-01 _huc14 _0-15
done 0 2019-10-01 2020-05-01 _huc14 _0-15
done 1 2019-10-01 2020-05-01 _huc14 _0-15
done 2 2019-10-01 2020-05-01 _huc14 _0-15
done 3 2019-10-01 2020-05-01 _huc14 _0-15
done 4 2019-10-01 2020-05-01 _huc14 _0-15
done 5 2019-10-01 2020-05-01 _huc14 _0-15
done 6 2019-10-01 2020-05-01 _huc14 _0-15
done 7 2019-10-01 2020-05-01 _huc14 _0-15
done 8 2019-10-01 2020-05-01 _huc14 _0-15
done 9 2019-10-01 2020-05-01 _huc14 _0-15
done 10 2019-10-01 2020-05-01 _huc14 _0-15
done 11 2019-10-01 2020-05-01 _huc14 _0-15
done 12 2019-10-01 2020-05-01 _huc14 _0-15
done 13 2019-10-01 2020-05-01 _huc14 _0-15
done 14 2019-10-01 2020-05-01 _huc14 _0-15
done 0 2020-04-01 2020-11-01 _huc14 _0-15
done 1 2020-04-01 2020-11-01 _huc14 _0-15
done 2 2020-04-01 2020-11-01 _huc14 _0-15
done 3 2020-04-01 2020-11-01 _huc14 _0-15
done 4 2020-04-01 2020-11-01 _huc14 _0-15
done 5 2020-04-01 2020-11-01 _huc14 _0-15
done 6 2020-04-01 2020-11-01 _huc14 _0-15
done 7 2020-04-01 2020-11-01

In [7]:
###HUC14
#70 Sites

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc14_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(15,30)
huc = '_huc14'
siteSpan = '_15-30'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc14 _15-30
done 1 2013-03-18 2013-11-01 _huc14 _15-30
done 2 2013-03-18 2013-11-01 _huc14 _15-30
done 3 2013-03-18 2013-11-01 _huc14 _15-30
done 4 2013-03-18 2013-11-01 _huc14 _15-30
done 5 2013-03-18 2013-11-01 _huc14 _15-30
done 6 2013-03-18 2013-11-01 _huc14 _15-30
done 7 2013-03-18 2013-11-01 _huc14 _15-30
done 8 2013-03-18 2013-11-01 _huc14 _15-30
done 9 2013-03-18 2013-11-01 _huc14 _15-30
done 10 2013-03-18 2013-11-01 _huc14 _15-30
done 11 2013-03-18 2013-11-01 _huc14 _15-30
done 12 2013-03-18 2013-11-01 _huc14 _15-30
done 13 2013-03-18 2013-11-01 _huc14 _15-30
done 14 2013-03-18 2013-11-01 _huc14 _15-30
done 0 2013-10-01 2014-05-01 _huc14 _15-30
done 1 2013-10-01 2014-05-01 _huc14 _15-30
done 2 2013-10-01 2014-05-01 _huc14 _15-30
done 3 2013-10-01 2014-05-01 _huc14 _15-30
done 4 2013-10-01 2014-05-01 _huc14 _15-30
done 5 2013-10-01 2014-05-01 _huc14 _15-30
done 6 2013-10-01 2014-05-01 _huc14 _15-30
done 7 2013-10-01 2014-05-01 _huc14 _15-30
done 8

done 10 2019-04-01 2019-11-01 _huc14 _15-30
done 11 2019-04-01 2019-11-01 _huc14 _15-30
done 12 2019-04-01 2019-11-01 _huc14 _15-30
done 13 2019-04-01 2019-11-01 _huc14 _15-30
done 14 2019-04-01 2019-11-01 _huc14 _15-30
done 0 2019-10-01 2020-05-01 _huc14 _15-30
done 1 2019-10-01 2020-05-01 _huc14 _15-30
done 2 2019-10-01 2020-05-01 _huc14 _15-30
done 3 2019-10-01 2020-05-01 _huc14 _15-30
done 4 2019-10-01 2020-05-01 _huc14 _15-30
done 5 2019-10-01 2020-05-01 _huc14 _15-30
done 6 2019-10-01 2020-05-01 _huc14 _15-30
done 7 2019-10-01 2020-05-01 _huc14 _15-30
done 8 2019-10-01 2020-05-01 _huc14 _15-30
done 9 2019-10-01 2020-05-01 _huc14 _15-30
done 10 2019-10-01 2020-05-01 _huc14 _15-30
done 11 2019-10-01 2020-05-01 _huc14 _15-30
done 12 2019-10-01 2020-05-01 _huc14 _15-30
done 13 2019-10-01 2020-05-01 _huc14 _15-30
done 14 2019-10-01 2020-05-01 _huc14 _15-30
done 0 2020-04-01 2020-11-01 _huc14 _15-30
done 1 2020-04-01 2020-11-01 _huc14 _15-30
done 2 2020-04-01 2020-11-01 _huc14 _15-30
d

In [11]:
###HUC14
#70 Sites

datesStag = (('2020-04-01', '2020-11-01'),
        ('2020-10-01', '2021-05-01'),
        ('2021-04-01', '2021-11-01'),
        ('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc14_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(30,45)
huc = '_huc14'
siteSpan = '_30-45'

#Call function
featTest(hucList)

done 0 2020-04-01 2020-11-01 _huc14 _30-45
done 1 2020-04-01 2020-11-01 _huc14 _30-45
done 2 2020-04-01 2020-11-01 _huc14 _30-45
done 3 2020-04-01 2020-11-01 _huc14 _30-45
done 4 2020-04-01 2020-11-01 _huc14 _30-45
done 5 2020-04-01 2020-11-01 _huc14 _30-45
done 6 2020-04-01 2020-11-01 _huc14 _30-45
done 7 2020-04-01 2020-11-01 _huc14 _30-45
done 8 2020-04-01 2020-11-01 _huc14 _30-45
done 9 2020-04-01 2020-11-01 _huc14 _30-45
done 10 2020-04-01 2020-11-01 _huc14 _30-45
done 11 2020-04-01 2020-11-01 _huc14 _30-45
done 12 2020-04-01 2020-11-01 _huc14 _30-45
done 13 2020-04-01 2020-11-01 _huc14 _30-45
done 14 2020-04-01 2020-11-01 _huc14 _30-45
done 0 2020-10-01 2021-05-01 _huc14 _30-45
done 1 2020-10-01 2021-05-01 _huc14 _30-45
done 2 2020-10-01 2021-05-01 _huc14 _30-45
done 3 2020-10-01 2021-05-01 _huc14 _30-45
done 4 2020-10-01 2021-05-01 _huc14 _30-45
done 5 2020-10-01 2021-05-01 _huc14 _30-45
done 6 2020-10-01 2021-05-01 _huc14 _30-45
done 7 2020-10-01 2021-05-01 _huc14 _30-45
done 8

In [15]:
###HUC14
#70 Sites
datesStag = (('2016-10-01', '2017-05-01'),
        ('2017-04-01', '2017-11-01'),
        ('2017-10-01', '2018-05-01'),
        ('2018-04-01', '2018-11-01'),
        ('2018-10-01', '2019-05-01'),
        ('2019-04-01', '2019-11-01'),
        ('2019-10-01', '2020-05-01'),
        ('2020-04-01', '2020-11-01'),
        ('2020-10-01', '2021-05-01'),
        ('2021-04-01', '2021-11-01'),
        ('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc14_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(45,60)
huc = '_huc14'
siteSpan = '_45-60'

#Call function
featTest(hucList)

done 0 2016-10-01 2017-05-01 _huc14 _45-60
done 1 2016-10-01 2017-05-01 _huc14 _45-60
done 2 2016-10-01 2017-05-01 _huc14 _45-60
done 3 2016-10-01 2017-05-01 _huc14 _45-60
done 4 2016-10-01 2017-05-01 _huc14 _45-60
done 5 2016-10-01 2017-05-01 _huc14 _45-60
done 6 2016-10-01 2017-05-01 _huc14 _45-60
done 7 2016-10-01 2017-05-01 _huc14 _45-60
done 8 2016-10-01 2017-05-01 _huc14 _45-60
done 9 2016-10-01 2017-05-01 _huc14 _45-60
done 10 2016-10-01 2017-05-01 _huc14 _45-60
done 11 2016-10-01 2017-05-01 _huc14 _45-60
done 12 2016-10-01 2017-05-01 _huc14 _45-60
done 13 2016-10-01 2017-05-01 _huc14 _45-60
done 14 2016-10-01 2017-05-01 _huc14 _45-60
done 0 2017-04-01 2017-11-01 _huc14 _45-60
done 1 2017-04-01 2017-11-01 _huc14 _45-60
done 2 2017-04-01 2017-11-01 _huc14 _45-60
done 3 2017-04-01 2017-11-01 _huc14 _45-60
done 4 2017-04-01 2017-11-01 _huc14 _45-60
done 5 2017-04-01 2017-11-01 _huc14 _45-60
done 6 2017-04-01 2017-11-01 _huc14 _45-60
done 7 2017-04-01 2017-11-01 _huc14 _45-60
done 8

done 10 2022-10-01 2023-03-01 _huc14 _45-60
done 11 2022-10-01 2023-03-01 _huc14 _45-60
done 12 2022-10-01 2023-03-01 _huc14 _45-60
done 13 2022-10-01 2023-03-01 _huc14 _45-60
done 14 2022-10-01 2023-03-01 _huc14 _45-60


In [17]:
###HUC14
#70 Sites

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc14_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(60,70)
huc = '_huc14'
siteSpan = '_60-70'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc14 _60-70
done 1 2013-03-18 2013-11-01 _huc14 _60-70
done 2 2013-03-18 2013-11-01 _huc14 _60-70
done 3 2013-03-18 2013-11-01 _huc14 _60-70
done 4 2013-03-18 2013-11-01 _huc14 _60-70
done 5 2013-03-18 2013-11-01 _huc14 _60-70
done 6 2013-03-18 2013-11-01 _huc14 _60-70
done 7 2013-03-18 2013-11-01 _huc14 _60-70
done 8 2013-03-18 2013-11-01 _huc14 _60-70
done 9 2013-03-18 2013-11-01 _huc14 _60-70
done 0 2013-10-01 2014-05-01 _huc14 _60-70
done 1 2013-10-01 2014-05-01 _huc14 _60-70
done 2 2013-10-01 2014-05-01 _huc14 _60-70
done 3 2013-10-01 2014-05-01 _huc14 _60-70
done 4 2013-10-01 2014-05-01 _huc14 _60-70
done 5 2013-10-01 2014-05-01 _huc14 _60-70
done 6 2013-10-01 2014-05-01 _huc14 _60-70
done 7 2013-10-01 2014-05-01 _huc14 _60-70
done 8 2013-10-01 2014-05-01 _huc14 _60-70
done 9 2013-10-01 2014-05-01 _huc14 _60-70
done 0 2014-04-01 2014-11-01 _huc14 _60-70
done 1 2014-04-01 2014-11-01 _huc14 _60-70
done 2 2014-04-01 2014-11-01 _huc14 _60-70
done 3 2014

done 1 2022-10-01 2023-03-01 _huc14 _60-70
done 2 2022-10-01 2023-03-01 _huc14 _60-70
done 3 2022-10-01 2023-03-01 _huc14 _60-70
done 4 2022-10-01 2023-03-01 _huc14 _60-70
done 5 2022-10-01 2023-03-01 _huc14 _60-70
done 6 2022-10-01 2023-03-01 _huc14 _60-70
done 7 2022-10-01 2023-03-01 _huc14 _60-70
done 8 2022-10-01 2023-03-01 _huc14 _60-70
done 9 2022-10-01 2023-03-01 _huc14 _60-70


In [8]:
###HUC18
#48 Sites

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc18_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(0,12)
huc = '_huc18'
siteSpan = '_0-12'

#Call function
featTest(hucList)

done 0 2020-04-01 2020-11-01 _huc18 _0-12
done 1 2020-04-01 2020-11-01 _huc18 _0-12
done 2 2020-04-01 2020-11-01 _huc18 _0-12
done 3 2020-04-01 2020-11-01 _huc18 _0-12
done 4 2020-04-01 2020-11-01 _huc18 _0-12
done 5 2020-04-01 2020-11-01 _huc18 _0-12
done 6 2020-04-01 2020-11-01 _huc18 _0-12
done 7 2020-04-01 2020-11-01 _huc18 _0-12
done 8 2020-04-01 2020-11-01 _huc18 _0-12
done 9 2020-04-01 2020-11-01 _huc18 _0-12
done 10 2020-04-01 2020-11-01 _huc18 _0-12
done 11 2020-04-01 2020-11-01 _huc18 _0-12
done 0 2020-10-01 2021-05-01 _huc18 _0-12
done 1 2020-10-01 2021-05-01 _huc18 _0-12
done 2 2020-10-01 2021-05-01 _huc18 _0-12
done 3 2020-10-01 2021-05-01 _huc18 _0-12
done 4 2020-10-01 2021-05-01 _huc18 _0-12
done 5 2020-10-01 2021-05-01 _huc18 _0-12
done 6 2020-10-01 2021-05-01 _huc18 _0-12
done 7 2020-10-01 2021-05-01 _huc18 _0-12
done 8 2020-10-01 2021-05-01 _huc18 _0-12
done 9 2020-10-01 2021-05-01 _huc18 _0-12
done 10 2020-10-01 2021-05-01 _huc18 _0-12
done 11 2020-10-01 2021-05-01 _

In [10]:
###HUC18
#48 Sites
#4/4/23

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc18_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(12,24)
huc = '_huc18'
siteSpan = '_12-24'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc18 _12-24
done 1 2013-03-18 2013-11-01 _huc18 _12-24
done 2 2013-03-18 2013-11-01 _huc18 _12-24
done 3 2013-03-18 2013-11-01 _huc18 _12-24
done 4 2013-03-18 2013-11-01 _huc18 _12-24
done 5 2013-03-18 2013-11-01 _huc18 _12-24
done 6 2013-03-18 2013-11-01 _huc18 _12-24
done 7 2013-03-18 2013-11-01 _huc18 _12-24
done 8 2013-03-18 2013-11-01 _huc18 _12-24
done 9 2013-03-18 2013-11-01 _huc18 _12-24
done 10 2013-03-18 2013-11-01 _huc18 _12-24
done 11 2013-03-18 2013-11-01 _huc18 _12-24
done 0 2013-10-01 2014-05-01 _huc18 _12-24
done 1 2013-10-01 2014-05-01 _huc18 _12-24
done 2 2013-10-01 2014-05-01 _huc18 _12-24
done 3 2013-10-01 2014-05-01 _huc18 _12-24
done 4 2013-10-01 2014-05-01 _huc18 _12-24
done 5 2013-10-01 2014-05-01 _huc18 _12-24
done 6 2013-10-01 2014-05-01 _huc18 _12-24
done 7 2013-10-01 2014-05-01 _huc18 _12-24
done 8 2013-10-01 2014-05-01 _huc18 _12-24
done 9 2013-10-01 2014-05-01 _huc18 _12-24
done 10 2013-10-01 2014-05-01 _huc18 _12-24
done 11 

done 10 2020-10-01 2021-05-01 _huc18 _12-24
done 11 2020-10-01 2021-05-01 _huc18 _12-24
done 0 2021-04-01 2021-11-01 _huc18 _12-24
done 1 2021-04-01 2021-11-01 _huc18 _12-24
done 2 2021-04-01 2021-11-01 _huc18 _12-24
done 3 2021-04-01 2021-11-01 _huc18 _12-24
done 4 2021-04-01 2021-11-01 _huc18 _12-24
done 5 2021-04-01 2021-11-01 _huc18 _12-24
done 6 2021-04-01 2021-11-01 _huc18 _12-24
done 7 2021-04-01 2021-11-01 _huc18 _12-24
done 8 2021-04-01 2021-11-01 _huc18 _12-24
done 9 2021-04-01 2021-11-01 _huc18 _12-24
done 10 2021-04-01 2021-11-01 _huc18 _12-24
done 11 2021-04-01 2021-11-01 _huc18 _12-24
done 0 2021-10-01 2022-05-01 _huc18 _12-24
done 1 2021-10-01 2022-05-01 _huc18 _12-24
done 2 2021-10-01 2022-05-01 _huc18 _12-24
done 3 2021-10-01 2022-05-01 _huc18 _12-24
done 4 2021-10-01 2022-05-01 _huc18 _12-24
done 5 2021-10-01 2022-05-01 _huc18 _12-24
done 6 2021-10-01 2022-05-01 _huc18 _12-24
done 7 2021-10-01 2022-05-01 _huc18 _12-24
done 8 2021-10-01 2022-05-01 _huc18 _12-24
done 9 

In [12]:
###HUC18
#48 Sites

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc18_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(24,36)
huc = '_huc18'
siteSpan = '_24-36'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc18 _24-36
done 1 2013-03-18 2013-11-01 _huc18 _24-36
done 2 2013-03-18 2013-11-01 _huc18 _24-36
done 3 2013-03-18 2013-11-01 _huc18 _24-36
done 4 2013-03-18 2013-11-01 _huc18 _24-36
done 5 2013-03-18 2013-11-01 _huc18 _24-36
done 6 2013-03-18 2013-11-01 _huc18 _24-36
done 7 2013-03-18 2013-11-01 _huc18 _24-36
done 8 2013-03-18 2013-11-01 _huc18 _24-36
done 9 2013-03-18 2013-11-01 _huc18 _24-36
done 10 2013-03-18 2013-11-01 _huc18 _24-36
done 11 2013-03-18 2013-11-01 _huc18 _24-36
done 0 2013-10-01 2014-05-01 _huc18 _24-36
done 1 2013-10-01 2014-05-01 _huc18 _24-36
done 2 2013-10-01 2014-05-01 _huc18 _24-36
done 3 2013-10-01 2014-05-01 _huc18 _24-36
done 4 2013-10-01 2014-05-01 _huc18 _24-36
done 5 2013-10-01 2014-05-01 _huc18 _24-36
done 6 2013-10-01 2014-05-01 _huc18 _24-36
done 7 2013-10-01 2014-05-01 _huc18 _24-36
done 8 2013-10-01 2014-05-01 _huc18 _24-36
done 9 2013-10-01 2014-05-01 _huc18 _24-36
done 10 2013-10-01 2014-05-01 _huc18 _24-36
done 11 

done 10 2020-10-01 2021-05-01 _huc18 _24-36
done 11 2020-10-01 2021-05-01 _huc18 _24-36
done 0 2021-04-01 2021-11-01 _huc18 _24-36
done 1 2021-04-01 2021-11-01 _huc18 _24-36
done 2 2021-04-01 2021-11-01 _huc18 _24-36
done 3 2021-04-01 2021-11-01 _huc18 _24-36
done 4 2021-04-01 2021-11-01 _huc18 _24-36
done 5 2021-04-01 2021-11-01 _huc18 _24-36
done 6 2021-04-01 2021-11-01 _huc18 _24-36
done 7 2021-04-01 2021-11-01 _huc18 _24-36
done 8 2021-04-01 2021-11-01 _huc18 _24-36
done 9 2021-04-01 2021-11-01 _huc18 _24-36
done 10 2021-04-01 2021-11-01 _huc18 _24-36
done 11 2021-04-01 2021-11-01 _huc18 _24-36
done 0 2021-10-01 2022-05-01 _huc18 _24-36
done 1 2021-10-01 2022-05-01 _huc18 _24-36
done 2 2021-10-01 2022-05-01 _huc18 _24-36
done 3 2021-10-01 2022-05-01 _huc18 _24-36
done 4 2021-10-01 2022-05-01 _huc18 _24-36
done 5 2021-10-01 2022-05-01 _huc18 _24-36
done 6 2021-10-01 2022-05-01 _huc18 _24-36
done 7 2021-10-01 2022-05-01 _huc18 _24-36
done 8 2021-10-01 2022-05-01 _huc18 _24-36
done 9 

In [15]:
###HUC18
#48 Sites
datesStag = (('2018-04-01', '2018-11-01'),
        ('2018-10-01', '2019-05-01'),
        ('2019-04-01', '2019-11-01'),
        ('2019-10-01', '2020-05-01'),
        ('2020-04-01', '2020-11-01'),
        ('2020-10-01', '2021-05-01'),
        ('2021-04-01', '2021-11-01'),
        ('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))
#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc18_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(500).slice(36,48)
huc = '_huc18'
siteSpan = '_36-48'

#Call function
featTest(hucList)

done 0 2018-04-01 2018-11-01 _huc18 _36-48
done 1 2018-04-01 2018-11-01 _huc18 _36-48
done 2 2018-04-01 2018-11-01 _huc18 _36-48
done 3 2018-04-01 2018-11-01 _huc18 _36-48
done 4 2018-04-01 2018-11-01 _huc18 _36-48
done 5 2018-04-01 2018-11-01 _huc18 _36-48
done 6 2018-04-01 2018-11-01 _huc18 _36-48
done 7 2018-04-01 2018-11-01 _huc18 _36-48
done 8 2018-04-01 2018-11-01 _huc18 _36-48
done 9 2018-04-01 2018-11-01 _huc18 _36-48
done 10 2018-04-01 2018-11-01 _huc18 _36-48
done 11 2018-04-01 2018-11-01 _huc18 _36-48
done 0 2018-10-01 2019-05-01 _huc18 _36-48
done 1 2018-10-01 2019-05-01 _huc18 _36-48
done 2 2018-10-01 2019-05-01 _huc18 _36-48
done 3 2018-10-01 2019-05-01 _huc18 _36-48
done 4 2018-10-01 2019-05-01 _huc18 _36-48
done 5 2018-10-01 2019-05-01 _huc18 _36-48
done 6 2018-10-01 2019-05-01 _huc18 _36-48
done 7 2018-10-01 2019-05-01 _huc18 _36-48
done 8 2018-10-01 2019-05-01 _huc18 _36-48
done 9 2018-10-01 2019-05-01 _huc18 _36-48
done 10 2018-10-01 2019-05-01 _huc18 _36-48
done 11 

In [12]:
###HUC02
#30 Sites

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc02_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(2000).slice(0,10)
huc = '_huc2'
siteSpan = '_0-10'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc2 _0-10
done 1 2013-03-18 2013-11-01 _huc2 _0-10
done 2 2013-03-18 2013-11-01 _huc2 _0-10
done 3 2013-03-18 2013-11-01 _huc2 _0-10
done 4 2013-03-18 2013-11-01 _huc2 _0-10
done 5 2013-03-18 2013-11-01 _huc2 _0-10
done 6 2013-03-18 2013-11-01 _huc2 _0-10
done 7 2013-03-18 2013-11-01 _huc2 _0-10
done 8 2013-03-18 2013-11-01 _huc2 _0-10
done 9 2013-03-18 2013-11-01 _huc2 _0-10
done 0 2013-10-01 2014-05-01 _huc2 _0-10
done 1 2013-10-01 2014-05-01 _huc2 _0-10
done 2 2013-10-01 2014-05-01 _huc2 _0-10
done 3 2013-10-01 2014-05-01 _huc2 _0-10
done 4 2013-10-01 2014-05-01 _huc2 _0-10
done 5 2013-10-01 2014-05-01 _huc2 _0-10
done 6 2013-10-01 2014-05-01 _huc2 _0-10
done 7 2013-10-01 2014-05-01 _huc2 _0-10
done 8 2013-10-01 2014-05-01 _huc2 _0-10
done 9 2013-10-01 2014-05-01 _huc2 _0-10
done 0 2014-04-01 2014-11-01 _huc2 _0-10
done 1 2014-04-01 2014-11-01 _huc2 _0-10
done 2 2014-04-01 2014-11-01 _huc2 _0-10
done 3 2014-04-01 2014-11-01 _huc2 _0-10
done 4 2014-04-0

In [14]:
###HUC02
#30 Sites

datesStag = (('2016-04-01', '2016-11-01'),
        ('2016-10-01', '2017-05-01'),
        ('2017-04-01', '2017-11-01'),
        ('2017-10-01', '2018-05-01'),
        ('2018-04-01', '2018-11-01'),
        ('2018-10-01', '2019-05-01'),
        ('2019-04-01', '2019-11-01'),
        ('2019-10-01', '2020-05-01'),
        ('2020-04-01', '2020-11-01'),
        ('2020-10-01', '2021-05-01'),
        ('2021-04-01', '2021-11-01'),
        ('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc02_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(2000).slice(10,20)
huc = '_huc2'
siteSpan = '_10-20'


#Call function
featTest(hucList)


done 0 2016-04-01 2016-11-01 _huc2 _10-20
done 1 2016-04-01 2016-11-01 _huc2 _10-20
done 2 2016-04-01 2016-11-01 _huc2 _10-20
done 3 2016-04-01 2016-11-01 _huc2 _10-20
done 4 2016-04-01 2016-11-01 _huc2 _10-20
done 5 2016-04-01 2016-11-01 _huc2 _10-20
done 6 2016-04-01 2016-11-01 _huc2 _10-20
done 7 2016-04-01 2016-11-01 _huc2 _10-20
done 8 2016-04-01 2016-11-01 _huc2 _10-20
done 9 2016-04-01 2016-11-01 _huc2 _10-20
done 0 2016-10-01 2017-05-01 _huc2 _10-20
done 1 2016-10-01 2017-05-01 _huc2 _10-20
done 2 2016-10-01 2017-05-01 _huc2 _10-20
done 3 2016-10-01 2017-05-01 _huc2 _10-20
done 4 2016-10-01 2017-05-01 _huc2 _10-20
done 5 2016-10-01 2017-05-01 _huc2 _10-20
done 6 2016-10-01 2017-05-01 _huc2 _10-20
done 7 2016-10-01 2017-05-01 _huc2 _10-20
done 8 2016-10-01 2017-05-01 _huc2 _10-20
done 9 2016-10-01 2017-05-01 _huc2 _10-20
done 0 2017-04-01 2017-11-01 _huc2 _10-20
done 1 2017-04-01 2017-11-01 _huc2 _10-20
done 2 2017-04-01 2017-11-01 _huc2 _10-20
done 3 2017-04-01 2017-11-01 _huc2

In [2]:
###HUC02
#30 Sites

datesStag = (('2016-04-01', '2016-11-01'),
        ('2016-10-01', '2017-05-01'),
        ('2017-04-01', '2017-11-01'),
        ('2017-10-01', '2018-05-01'),
        ('2018-04-01', '2018-11-01'),
        ('2018-10-01', '2019-05-01'),
        ('2019-04-01', '2019-11-01'),
        ('2019-10-01', '2020-05-01'),
        ('2020-04-01', '2020-11-01'),
        ('2020-10-01', '2021-05-01'),
        ('2021-04-01', '2021-11-01'),
        ('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc02_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(2000).slice(20,30)
huc = '_huc2'
siteSpan = '_20-30'


#Call function
featTest(hucList)


done 0 2016-04-01 2016-11-01 _huc2 _20-30
done 1 2016-04-01 2016-11-01 _huc2 _20-30
done 2 2016-04-01 2016-11-01 _huc2 _20-30
done 3 2016-04-01 2016-11-01 _huc2 _20-30
done 4 2016-04-01 2016-11-01 _huc2 _20-30
done 5 2016-04-01 2016-11-01 _huc2 _20-30
done 6 2016-04-01 2016-11-01 _huc2 _20-30
done 7 2016-04-01 2016-11-01 _huc2 _20-30
done 8 2016-04-01 2016-11-01 _huc2 _20-30
done 9 2016-04-01 2016-11-01 _huc2 _20-30
done 0 2016-10-01 2017-05-01 _huc2 _20-30
done 1 2016-10-01 2017-05-01 _huc2 _20-30
done 2 2016-10-01 2017-05-01 _huc2 _20-30
done 3 2016-10-01 2017-05-01 _huc2 _20-30
done 4 2016-10-01 2017-05-01 _huc2 _20-30
done 5 2016-10-01 2017-05-01 _huc2 _20-30
done 6 2016-10-01 2017-05-01 _huc2 _20-30
done 7 2016-10-01 2017-05-01 _huc2 _20-30
done 8 2016-10-01 2017-05-01 _huc2 _20-30
done 9 2016-10-01 2017-05-01 _huc2 _20-30
done 0 2017-04-01 2017-11-01 _huc2 _20-30
done 1 2017-04-01 2017-11-01 _huc2 _20-30
done 2 2017-04-01 2017-11-01 _huc2 _20-30
done 3 2017-04-01 2017-11-01 _huc2

In [10]:
###HUC16
#11 Sites

datesStag = (('2014-10-01', '2015-05-01'),
        ('2015-04-01', '2015-11-01'),
        ('2015-10-01', '2016-05-01'),
        ('2016-04-01', '2016-11-01'),
        ('2016-10-01', '2017-05-01'),
        ('2017-04-01', '2017-11-01'),
        ('2017-10-01', '2018-05-01'),
        ('2018-04-01', '2018-11-01'),
        ('2018-10-01', '2019-05-01'),
        ('2019-04-01', '2019-11-01'),
        ('2019-10-01', '2020-05-01'),
        ('2020-04-01', '2020-11-01'),
        ('2020-10-01', '2021-05-01'),
        ('2021-04-01', '2021-11-01'),
        ('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc16_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(2000)
huc = '_huc16'
siteSpan = '_0-11'


#Call function
featTest(hucList)


done 0 2014-10-01 2015-05-01 _huc16 _0-11
done 1 2014-10-01 2015-05-01 _huc16 _0-11
done 2 2014-10-01 2015-05-01 _huc16 _0-11
done 3 2014-10-01 2015-05-01 _huc16 _0-11
done 4 2014-10-01 2015-05-01 _huc16 _0-11
done 5 2014-10-01 2015-05-01 _huc16 _0-11
done 6 2014-10-01 2015-05-01 _huc16 _0-11
done 7 2014-10-01 2015-05-01 _huc16 _0-11
done 8 2014-10-01 2015-05-01 _huc16 _0-11
done 9 2014-10-01 2015-05-01 _huc16 _0-11
done 10 2014-10-01 2015-05-01 _huc16 _0-11
done 0 2015-04-01 2015-11-01 _huc16 _0-11
done 1 2015-04-01 2015-11-01 _huc16 _0-11
done 2 2015-04-01 2015-11-01 _huc16 _0-11
done 3 2015-04-01 2015-11-01 _huc16 _0-11
done 4 2015-04-01 2015-11-01 _huc16 _0-11
done 5 2015-04-01 2015-11-01 _huc16 _0-11
done 6 2015-04-01 2015-11-01 _huc16 _0-11
done 7 2015-04-01 2015-11-01 _huc16 _0-11
done 8 2015-04-01 2015-11-01 _huc16 _0-11
done 9 2015-04-01 2015-11-01 _huc16 _0-11
done 10 2015-04-01 2015-11-01 _huc16 _0-11
done 0 2015-10-01 2016-05-01 _huc16 _0-11
done 1 2015-10-01 2016-05-01 _hu

In [5]:
###HUC06
#15 Sites
#9 more

huc6SitesFinished = ee.List(['TDECWR_WQX-TNW000007005', '21AWIC-7430', 'TDECWPC-TENNE568.2RO', '21AWIC-135', 
                             '21AWIC-124', '21AWIC-49', '21AWIC-142', '21AWIC-154', '21AWIC-3872', '21AWIC-168', 
                             '21AWIC-123', '11NPSWRD_WQX-SHIL_PCA_205.5', '11NPSWRD_WQX-SHIL_PCA_205.2', 
                             '21AWIC-133', '11NPSWRD_WQX-SHIL_PCA_203.8', '21AWIC-122', 
                             '11NPSWRD_WQX-SHIL_PCA_202.2'])

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc06_5sitelatlong_2')
hucSites = hucSites.filter(ee.Filter.inList('SiteID', huc6SitesFinished).Not())
#print(hucSites.aggregate_array('SiteID').getInfo())
hucList = ee.FeatureCollection(hucSites).toList(2000)
huc = '_huc6'
siteSpan = '_0-9'


#Call function
featTest(hucList)


done 0 2013-03-18 2013-11-01 _huc6 _0-9
done 1 2013-03-18 2013-11-01 _huc6 _0-9
done 2 2013-03-18 2013-11-01 _huc6 _0-9
done 3 2013-03-18 2013-11-01 _huc6 _0-9
done 4 2013-03-18 2013-11-01 _huc6 _0-9
done 5 2013-03-18 2013-11-01 _huc6 _0-9
done 6 2013-03-18 2013-11-01 _huc6 _0-9
done 7 2013-03-18 2013-11-01 _huc6 _0-9
done 8 2013-03-18 2013-11-01 _huc6 _0-9
done 0 2013-10-01 2014-05-01 _huc6 _0-9
done 1 2013-10-01 2014-05-01 _huc6 _0-9
done 2 2013-10-01 2014-05-01 _huc6 _0-9
done 3 2013-10-01 2014-05-01 _huc6 _0-9
done 4 2013-10-01 2014-05-01 _huc6 _0-9
done 5 2013-10-01 2014-05-01 _huc6 _0-9
done 6 2013-10-01 2014-05-01 _huc6 _0-9
done 7 2013-10-01 2014-05-01 _huc6 _0-9
done 8 2013-10-01 2014-05-01 _huc6 _0-9
done 0 2014-04-01 2014-11-01 _huc6 _0-9
done 1 2014-04-01 2014-11-01 _huc6 _0-9
done 2 2014-04-01 2014-11-01 _huc6 _0-9
done 3 2014-04-01 2014-11-01 _huc6 _0-9
done 4 2014-04-01 2014-11-01 _huc6 _0-9
done 5 2014-04-01 2014-11-01 _huc6 _0-9
done 6 2014-04-01 2014-11-01 _huc6 _0-9


In [8]:
import pandas as pd
import numpy as np

# create a sample DataFrame
df = pd.DataFrame({
    'col1': [1, 2, np.nan, 4],
    'col2': [5, np.nan, 7, 8]
})

# replace NaN values with the corresponding non-null value from the same column
df = df.fillna(method='ffill')  # use 'bfill' for backward filling

print(df)

   col1  col2
0   1.0   5.0
1   2.0   5.0
2   2.0   7.0
3   4.0   8.0


In [6]:
###HUC08
#57 Sites

datesStag = (('2017-10-01', '2018-05-01'),
        ('2018-04-01', '2018-11-01'),
        ('2018-10-01', '2019-05-01'),
        ('2019-04-01', '2019-11-01'),
        ('2019-10-01', '2020-05-01'),
        ('2020-04-01', '2020-11-01'),
        ('2020-10-01', '2021-05-01'),
        ('2021-04-01', '2021-11-01'),
        ('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))



huc8SitesFinished = ee.List(['ARDEQH2O_WQX-OUA0037', 'ARDEQH2O_WQX-OUA0006', 'ARDEQH2O_WQX-FRA0013',
 'ARDEQH2O_WQX-OUA0030', 'ARDEQH2O_WQX-WHI0031', 'ARDEQH2O_WQX-WHI0036',
 'USGS-07288700', 'USGS-322023090544500', 'ARDEQH2O_WQX-OUA0008B'])

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc08_5sitelatlong_2')
hucSites = hucSites.filter(ee.Filter.inList('SiteID', huc8SitesFinished).Not())
#print(hucSites.aggregate_array('SiteID').size().getInfo())
hucList = ee.FeatureCollection(hucSites).toList(2000).slice(0,15)
huc = '_huc8'


#Call function
featTest(hucList)


done 0 2017-10-01 2018-05-01 _huc8
done 1 2017-10-01 2018-05-01 _huc8
done 2 2017-10-01 2018-05-01 _huc8
done 3 2017-10-01 2018-05-01 _huc8
done 4 2017-10-01 2018-05-01 _huc8
done 5 2017-10-01 2018-05-01 _huc8
done 6 2017-10-01 2018-05-01 _huc8
done 7 2017-10-01 2018-05-01 _huc8
done 8 2017-10-01 2018-05-01 _huc8
done 9 2017-10-01 2018-05-01 _huc8
done 10 2017-10-01 2018-05-01 _huc8
done 11 2017-10-01 2018-05-01 _huc8
done 12 2017-10-01 2018-05-01 _huc8
done 13 2017-10-01 2018-05-01 _huc8
done 14 2017-10-01 2018-05-01 _huc8
done 0 2018-04-01 2018-11-01 _huc8
done 1 2018-04-01 2018-11-01 _huc8
done 2 2018-04-01 2018-11-01 _huc8
done 3 2018-04-01 2018-11-01 _huc8
done 4 2018-04-01 2018-11-01 _huc8
done 5 2018-04-01 2018-11-01 _huc8
done 6 2018-04-01 2018-11-01 _huc8
done 7 2018-04-01 2018-11-01 _huc8
done 8 2018-04-01 2018-11-01 _huc8
done 9 2018-04-01 2018-11-01 _huc8
done 10 2018-04-01 2018-11-01 _huc8
done 11 2018-04-01 2018-11-01 _huc8
done 12 2018-04-01 2018-11-01 _huc8
done 13 2018

In [8]:
datesStag = (('2013-03-18', '2013-11-01'),
        ('2013-10-01', '2014-05-01'),
        ('2014-04-01', '2014-11-01'),
        ('2014-10-01', '2015-05-01'),
        ('2015-04-01', '2015-11-01'),
        ('2015-10-01', '2016-05-01'),
        ('2016-04-01', '2016-11-01'),
        ('2016-10-01', '2017-05-01'),
        ('2017-04-01', '2017-11-01'),
        ('2017-10-01', '2018-05-01'),
        ('2018-04-01', '2018-11-01'),
        ('2018-10-01', '2019-05-01'),
        ('2019-04-01', '2019-11-01'),
        ('2019-10-01', '2020-05-01'),
        ('2020-04-01', '2020-11-01'),
        ('2020-10-01', '2021-05-01'),
        ('2021-04-01', '2021-11-01'),
        ('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))

huc8SitesFinished = ee.List(['ARDEQH2O_WQX-OUA0037', 'ARDEQH2O_WQX-OUA0006', 'ARDEQH2O_WQX-FRA0013',
 'ARDEQH2O_WQX-OUA0030', 'ARDEQH2O_WQX-WHI0031', 'ARDEQH2O_WQX-WHI0036',
 'USGS-07288700', 'USGS-322023090544500', 'ARDEQH2O_WQX-OUA0008B'])

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc08_5sitelatlong_2')
hucSites = hucSites.filter(ee.Filter.inList('SiteID', huc8SitesFinished).Not())
#print(hucSites.aggregate_array('SiteID').size().getInfo())
hucList = ee.FeatureCollection(hucSites).toList(2000).slice(15,30)
huc = '_huc8'
siteSpan = '_15-30'

#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc8
done 1 2013-03-18 2013-11-01 _huc8
done 2 2013-03-18 2013-11-01 _huc8
done 3 2013-03-18 2013-11-01 _huc8
done 4 2013-03-18 2013-11-01 _huc8
done 5 2013-03-18 2013-11-01 _huc8
done 6 2013-03-18 2013-11-01 _huc8
done 7 2013-03-18 2013-11-01 _huc8
done 8 2013-03-18 2013-11-01 _huc8
done 9 2013-03-18 2013-11-01 _huc8
done 10 2013-03-18 2013-11-01 _huc8
done 11 2013-03-18 2013-11-01 _huc8
done 12 2013-03-18 2013-11-01 _huc8
done 13 2013-03-18 2013-11-01 _huc8
done 14 2013-03-18 2013-11-01 _huc8
done 0 2013-10-01 2014-05-01 _huc8
done 1 2013-10-01 2014-05-01 _huc8
done 2 2013-10-01 2014-05-01 _huc8
done 3 2013-10-01 2014-05-01 _huc8
done 4 2013-10-01 2014-05-01 _huc8
done 5 2013-10-01 2014-05-01 _huc8
done 6 2013-10-01 2014-05-01 _huc8
done 7 2013-10-01 2014-05-01 _huc8
done 8 2013-10-01 2014-05-01 _huc8
done 9 2013-10-01 2014-05-01 _huc8
done 10 2013-10-01 2014-05-01 _huc8
done 11 2013-10-01 2014-05-01 _huc8
done 12 2013-10-01 2014-05-01 _huc8
done 13 2013

done 7 2020-10-01 2021-05-01 _huc8
done 8 2020-10-01 2021-05-01 _huc8
done 9 2020-10-01 2021-05-01 _huc8
done 10 2020-10-01 2021-05-01 _huc8
done 11 2020-10-01 2021-05-01 _huc8
done 12 2020-10-01 2021-05-01 _huc8
done 13 2020-10-01 2021-05-01 _huc8
done 14 2020-10-01 2021-05-01 _huc8
done 0 2021-04-01 2021-11-01 _huc8
done 1 2021-04-01 2021-11-01 _huc8
done 2 2021-04-01 2021-11-01 _huc8
done 3 2021-04-01 2021-11-01 _huc8
done 4 2021-04-01 2021-11-01 _huc8
done 5 2021-04-01 2021-11-01 _huc8
done 6 2021-04-01 2021-11-01 _huc8
done 7 2021-04-01 2021-11-01 _huc8
done 8 2021-04-01 2021-11-01 _huc8
done 9 2021-04-01 2021-11-01 _huc8
done 10 2021-04-01 2021-11-01 _huc8
done 11 2021-04-01 2021-11-01 _huc8
done 12 2021-04-01 2021-11-01 _huc8
done 13 2021-04-01 2021-11-01 _huc8
done 14 2021-04-01 2021-11-01 _huc8
done 0 2021-10-01 2022-05-01 _huc8
done 1 2021-10-01 2022-05-01 _huc8
done 2 2021-10-01 2022-05-01 _huc8
done 3 2021-10-01 2022-05-01 _huc8
done 4 2021-10-01 2022-05-01 _huc8
done 5 202

In [13]:
huc8SitesFinished = ee.List(['ARDEQH2O_WQX-OUA0037', 'ARDEQH2O_WQX-OUA0006', 'ARDEQH2O_WQX-FRA0013',
 'ARDEQH2O_WQX-OUA0030', 'ARDEQH2O_WQX-WHI0031', 'ARDEQH2O_WQX-WHI0036',
 'USGS-07288700', 'USGS-322023090544500', 'ARDEQH2O_WQX-OUA0008B'])

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc08_5sitelatlong_2')
hucSites = hucSites.filter(ee.Filter.inList('SiteID', huc8SitesFinished).Not())
#print(hucSites.aggregate_array('SiteID').size().getInfo())
hucList = ee.FeatureCollection(hucSites).toList(2000).slice(30,39)
huc = '_huc8'
siteSpan = '_30-39'


#Call function
featTest(hucList)

done 0 2013-03-18 2013-11-01 _huc8 _30-39
done 1 2013-03-18 2013-11-01 _huc8 _30-39
done 2 2013-03-18 2013-11-01 _huc8 _30-39
done 3 2013-03-18 2013-11-01 _huc8 _30-39
done 4 2013-03-18 2013-11-01 _huc8 _30-39
done 5 2013-03-18 2013-11-01 _huc8 _30-39
done 6 2013-03-18 2013-11-01 _huc8 _30-39
done 7 2013-03-18 2013-11-01 _huc8 _30-39
done 8 2013-03-18 2013-11-01 _huc8 _30-39
done 0 2013-10-01 2014-05-01 _huc8 _30-39
done 1 2013-10-01 2014-05-01 _huc8 _30-39
done 2 2013-10-01 2014-05-01 _huc8 _30-39
done 3 2013-10-01 2014-05-01 _huc8 _30-39
done 4 2013-10-01 2014-05-01 _huc8 _30-39
done 5 2013-10-01 2014-05-01 _huc8 _30-39
done 6 2013-10-01 2014-05-01 _huc8 _30-39
done 7 2013-10-01 2014-05-01 _huc8 _30-39
done 8 2013-10-01 2014-05-01 _huc8 _30-39
done 0 2014-04-01 2014-11-01 _huc8 _30-39
done 1 2014-04-01 2014-11-01 _huc8 _30-39
done 2 2014-04-01 2014-11-01 _huc8 _30-39
done 3 2014-04-01 2014-11-01 _huc8 _30-39
done 4 2014-04-01 2014-11-01 _huc8 _30-39
done 5 2014-04-01 2014-11-01 _huc8

In [15]:
datesStag = (('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))

huc8SitesFinished = ee.List(['ARDEQH2O_WQX-OUA0037', 'ARDEQH2O_WQX-OUA0006', 'ARDEQH2O_WQX-FRA0013',
 'ARDEQH2O_WQX-OUA0030', 'ARDEQH2O_WQX-WHI0031', 'ARDEQH2O_WQX-WHI0036',
 'USGS-07288700', 'USGS-322023090544500', 'ARDEQH2O_WQX-OUA0008B'])

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc08_5sitelatlong_2')
hucSites = hucSites.filter(ee.Filter.inList('SiteID', huc8SitesFinished).Not())
#print(hucSites.aggregate_array('SiteID').size().getInfo())
hucList = ee.FeatureCollection(hucSites).toList(2000).slice(39,48)
huc = '_huc8'
siteSpan = '_39-48'


#Call function
featTest(hucList)

done 0 2021-10-01 2022-05-01 _huc8 _39-48
done 1 2021-10-01 2022-05-01 _huc8 _39-48
done 2 2021-10-01 2022-05-01 _huc8 _39-48
done 3 2021-10-01 2022-05-01 _huc8 _39-48
done 4 2021-10-01 2022-05-01 _huc8 _39-48
done 5 2021-10-01 2022-05-01 _huc8 _39-48
done 6 2021-10-01 2022-05-01 _huc8 _39-48
done 7 2021-10-01 2022-05-01 _huc8 _39-48
done 8 2021-10-01 2022-05-01 _huc8 _39-48
done 0 2022-04-01 2022-11-01 _huc8 _39-48
done 1 2022-04-01 2022-11-01 _huc8 _39-48
done 2 2022-04-01 2022-11-01 _huc8 _39-48
done 3 2022-04-01 2022-11-01 _huc8 _39-48
done 4 2022-04-01 2022-11-01 _huc8 _39-48
done 5 2022-04-01 2022-11-01 _huc8 _39-48
done 6 2022-04-01 2022-11-01 _huc8 _39-48
done 7 2022-04-01 2022-11-01 _huc8 _39-48
done 8 2022-04-01 2022-11-01 _huc8 _39-48
done 0 2022-10-01 2023-03-01 _huc8 _39-48
done 1 2022-10-01 2023-03-01 _huc8 _39-48
done 2 2022-10-01 2023-03-01 _huc8 _39-48
done 3 2022-10-01 2023-03-01 _huc8 _39-48
done 4 2022-10-01 2023-03-01 _huc8 _39-48
done 5 2022-10-01 2023-03-01 _huc8

In [26]:
###HUC12
#11 Sites

datesStag = (('2013-03-18', '2013-11-01'),
        ('2013-10-01', '2014-05-01'),
        ('2014-04-01', '2014-11-01'),
        ('2014-10-01', '2015-05-01'),
        ('2015-04-01', '2015-11-01'),
        ('2015-10-01', '2016-05-01'),
        ('2016-04-01', '2016-11-01'),
        ('2016-10-01', '2017-05-01'),
        ('2017-04-01', '2017-11-01'),
        ('2017-10-01', '2018-05-01'),
        ('2018-04-01', '2018-11-01'),
        ('2018-10-01', '2019-05-01'),
        ('2019-04-01', '2019-11-01'),
        ('2019-10-01', '2020-05-01'),
        ('2020-04-01', '2020-11-01'),
        ('2020-10-01', '2021-05-01'),
        ('2021-04-01', '2021-11-01'),
        ('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc12_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(2000)
huc = 'huc12'


#Call function
featTest(hucList)



done 0 2013-03-18 2013-11-01 huc12
done 1 2013-03-18 2013-11-01 huc12
done 2 2013-03-18 2013-11-01 huc12
done 3 2013-03-18 2013-11-01 huc12
done 4 2013-03-18 2013-11-01 huc12
done 5 2013-03-18 2013-11-01 huc12
done 6 2013-03-18 2013-11-01 huc12
done 7 2013-03-18 2013-11-01 huc12
done 8 2013-03-18 2013-11-01 huc12
done 9 2013-03-18 2013-11-01 huc12
done 10 2013-03-18 2013-11-01 huc12
done 0 2013-10-01 2014-05-01 huc12
done 1 2013-10-01 2014-05-01 huc12
done 2 2013-10-01 2014-05-01 huc12
done 3 2013-10-01 2014-05-01 huc12
done 4 2013-10-01 2014-05-01 huc12
done 5 2013-10-01 2014-05-01 huc12
done 6 2013-10-01 2014-05-01 huc12
done 7 2013-10-01 2014-05-01 huc12
done 8 2013-10-01 2014-05-01 huc12
done 9 2013-10-01 2014-05-01 huc12
done 10 2013-10-01 2014-05-01 huc12
done 0 2014-04-01 2014-11-01 huc12
done 1 2014-04-01 2014-11-01 huc12
done 2 2014-04-01 2014-11-01 huc12
done 3 2014-04-01 2014-11-01 huc12
done 4 2014-04-01 2014-11-01 huc12
done 5 2014-04-01 2014-11-01 huc12
done 6 2014-04-01 

In [12]:
#####HUC7
#47 Sites
datesStag = (('2017-04-01', '2017-11-01'),
        ('2017-10-01', '2018-05-01'),
        ('2018-04-01', '2018-11-01'),
        ('2018-10-01', '2019-05-01'),
        ('2019-04-01', '2019-11-01'),
        ('2019-10-01', '2020-05-01'),
        ('2020-04-01', '2020-11-01'),
        ('2020-10-01', '2021-05-01'),
        ('2021-04-01', '2021-11-01'),
        ('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc07_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(2000).slice(15,30)
huc = 'huc7'


#Call function
featTest(hucList)


done 0 2017-04-01 2017-11-01 huc7
done 1 2017-04-01 2017-11-01 huc7
done 2 2017-04-01 2017-11-01 huc7
done 3 2017-04-01 2017-11-01 huc7
done 4 2017-04-01 2017-11-01 huc7
done 5 2017-04-01 2017-11-01 huc7
done 6 2017-04-01 2017-11-01 huc7
done 7 2017-04-01 2017-11-01 huc7
done 8 2017-04-01 2017-11-01 huc7
done 9 2017-04-01 2017-11-01 huc7
done 10 2017-04-01 2017-11-01 huc7
done 11 2017-04-01 2017-11-01 huc7
done 12 2017-04-01 2017-11-01 huc7
done 13 2017-04-01 2017-11-01 huc7
done 14 2017-04-01 2017-11-01 huc7
done 0 2017-10-01 2018-05-01 huc7
done 1 2017-10-01 2018-05-01 huc7
done 2 2017-10-01 2018-05-01 huc7
done 3 2017-10-01 2018-05-01 huc7
done 4 2017-10-01 2018-05-01 huc7
done 5 2017-10-01 2018-05-01 huc7
done 6 2017-10-01 2018-05-01 huc7
done 7 2017-10-01 2018-05-01 huc7
done 8 2017-10-01 2018-05-01 huc7
done 9 2017-10-01 2018-05-01 huc7
done 10 2017-10-01 2018-05-01 huc7
done 11 2017-10-01 2018-05-01 huc7
done 12 2017-10-01 2018-05-01 huc7
done 13 2017-10-01 2018-05-01 huc7
done 

In [13]:
#####HUC7
#47 Sites
datesStag = (('2013-03-18', '2013-11-01'),
        ('2013-10-01', '2014-05-01'),
        ('2014-04-01', '2014-11-01'),
        ('2014-10-01', '2015-05-01'),
        ('2015-04-01', '2015-11-01'),
        ('2015-10-01', '2016-05-01'),
        ('2016-04-01', '2016-11-01'),
        ('2016-10-01', '2017-05-01'),
        ('2017-04-01', '2017-11-01'),
        ('2017-10-01', '2018-05-01'),
        ('2018-04-01', '2018-11-01'),
        ('2018-10-01', '2019-05-01'),
        ('2019-04-01', '2019-11-01'),
        ('2019-10-01', '2020-05-01'),
        ('2020-04-01', '2020-11-01'),
        ('2020-10-01', '2021-05-01'),
        ('2021-04-01', '2021-11-01'),
        ('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc07_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(2000).slice(30,47)
huc = 'huc7'


#Call function
featTest(hucList)


done 0 2013-03-18 2013-11-01 huc7
done 1 2013-03-18 2013-11-01 huc7
done 2 2013-03-18 2013-11-01 huc7
done 3 2013-03-18 2013-11-01 huc7
done 4 2013-03-18 2013-11-01 huc7
done 5 2013-03-18 2013-11-01 huc7
done 6 2013-03-18 2013-11-01 huc7
done 7 2013-03-18 2013-11-01 huc7
done 8 2013-03-18 2013-11-01 huc7
done 9 2013-03-18 2013-11-01 huc7
done 10 2013-03-18 2013-11-01 huc7
done 11 2013-03-18 2013-11-01 huc7
done 12 2013-03-18 2013-11-01 huc7
done 13 2013-03-18 2013-11-01 huc7
done 14 2013-03-18 2013-11-01 huc7
done 15 2013-03-18 2013-11-01 huc7
done 16 2013-03-18 2013-11-01 huc7
done 0 2013-10-01 2014-05-01 huc7
done 1 2013-10-01 2014-05-01 huc7
done 2 2013-10-01 2014-05-01 huc7
done 3 2013-10-01 2014-05-01 huc7
done 4 2013-10-01 2014-05-01 huc7
done 5 2013-10-01 2014-05-01 huc7
done 6 2013-10-01 2014-05-01 huc7
done 7 2013-10-01 2014-05-01 huc7
done 8 2013-10-01 2014-05-01 huc7
done 9 2013-10-01 2014-05-01 huc7
done 10 2013-10-01 2014-05-01 huc7
done 11 2013-10-01 2014-05-01 huc7
done 

done 1 2020-04-01 2020-11-01 huc7
done 2 2020-04-01 2020-11-01 huc7
done 3 2020-04-01 2020-11-01 huc7
done 4 2020-04-01 2020-11-01 huc7
done 5 2020-04-01 2020-11-01 huc7
done 6 2020-04-01 2020-11-01 huc7
done 7 2020-04-01 2020-11-01 huc7
done 8 2020-04-01 2020-11-01 huc7
done 9 2020-04-01 2020-11-01 huc7
done 10 2020-04-01 2020-11-01 huc7
done 11 2020-04-01 2020-11-01 huc7
done 12 2020-04-01 2020-11-01 huc7
done 13 2020-04-01 2020-11-01 huc7
done 14 2020-04-01 2020-11-01 huc7
done 15 2020-04-01 2020-11-01 huc7
done 16 2020-04-01 2020-11-01 huc7
done 0 2020-10-01 2021-05-01 huc7
done 1 2020-10-01 2021-05-01 huc7
done 2 2020-10-01 2021-05-01 huc7
done 3 2020-10-01 2021-05-01 huc7
done 4 2020-10-01 2021-05-01 huc7
done 5 2020-10-01 2021-05-01 huc7
done 6 2020-10-01 2021-05-01 huc7
done 7 2020-10-01 2021-05-01 huc7
done 8 2020-10-01 2021-05-01 huc7
done 9 2020-10-01 2021-05-01 huc7
done 10 2020-10-01 2021-05-01 huc7
done 11 2020-10-01 2021-05-01 huc7
done 12 2020-10-01 2021-05-01 huc7
done

In [14]:
#####HUC10
#124 Sites
datesStag = (('2013-03-18', '2013-11-01'),
        ('2013-10-01', '2014-05-01'),
        ('2014-04-01', '2014-11-01'),
        ('2014-10-01', '2015-05-01'),
        ('2015-04-01', '2015-11-01'),
        ('2015-10-01', '2016-05-01'),
        ('2016-04-01', '2016-11-01'),
        ('2016-10-01', '2017-05-01'),
        ('2017-04-01', '2017-11-01'),
        ('2017-10-01', '2018-05-01'),
        ('2018-04-01', '2018-11-01'),
        ('2018-10-01', '2019-05-01'),
        ('2019-04-01', '2019-11-01'),
        ('2019-10-01', '2020-05-01'),
        ('2020-04-01', '2020-11-01'),
        ('2020-10-01', '2021-05-01'),
        ('2021-04-01', '2021-11-01'),
        ('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc10_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(2000).slice(0,30)
huc = 'huc10'


#Call function
featTest(hucList)


done 0 2013-03-18 2013-11-01 huc10
done 1 2013-03-18 2013-11-01 huc10
done 2 2013-03-18 2013-11-01 huc10
done 3 2013-03-18 2013-11-01 huc10
done 4 2013-03-18 2013-11-01 huc10
done 5 2013-03-18 2013-11-01 huc10
done 6 2013-03-18 2013-11-01 huc10
done 7 2013-03-18 2013-11-01 huc10
done 8 2013-03-18 2013-11-01 huc10
done 9 2013-03-18 2013-11-01 huc10
done 10 2013-03-18 2013-11-01 huc10
done 11 2013-03-18 2013-11-01 huc10
done 12 2013-03-18 2013-11-01 huc10
done 13 2013-03-18 2013-11-01 huc10
done 14 2013-03-18 2013-11-01 huc10
done 15 2013-03-18 2013-11-01 huc10
done 16 2013-03-18 2013-11-01 huc10
done 17 2013-03-18 2013-11-01 huc10
done 18 2013-03-18 2013-11-01 huc10
done 19 2013-03-18 2013-11-01 huc10
done 20 2013-03-18 2013-11-01 huc10
done 21 2013-03-18 2013-11-01 huc10
done 22 2013-03-18 2013-11-01 huc10
done 23 2013-03-18 2013-11-01 huc10
done 24 2013-03-18 2013-11-01 huc10
done 25 2013-03-18 2013-11-01 huc10
done 26 2013-03-18 2013-11-01 huc10
done 27 2013-03-18 2013-11-01 huc10
do

done 20 2016-10-01 2017-05-01 huc10
done 21 2016-10-01 2017-05-01 huc10
done 22 2016-10-01 2017-05-01 huc10
done 23 2016-10-01 2017-05-01 huc10
done 24 2016-10-01 2017-05-01 huc10
done 25 2016-10-01 2017-05-01 huc10
done 26 2016-10-01 2017-05-01 huc10
done 27 2016-10-01 2017-05-01 huc10
done 28 2016-10-01 2017-05-01 huc10
done 29 2016-10-01 2017-05-01 huc10
done 0 2017-04-01 2017-11-01 huc10
done 1 2017-04-01 2017-11-01 huc10
done 2 2017-04-01 2017-11-01 huc10
done 3 2017-04-01 2017-11-01 huc10
done 4 2017-04-01 2017-11-01 huc10
done 5 2017-04-01 2017-11-01 huc10
done 6 2017-04-01 2017-11-01 huc10
done 7 2017-04-01 2017-11-01 huc10
done 8 2017-04-01 2017-11-01 huc10
done 9 2017-04-01 2017-11-01 huc10
done 10 2017-04-01 2017-11-01 huc10
done 11 2017-04-01 2017-11-01 huc10
done 12 2017-04-01 2017-11-01 huc10
done 13 2017-04-01 2017-11-01 huc10
done 14 2017-04-01 2017-11-01 huc10
done 15 2017-04-01 2017-11-01 huc10
done 16 2017-04-01 2017-11-01 huc10
done 17 2017-04-01 2017-11-01 huc10
do

done 10 2020-10-01 2021-05-01 huc10
done 11 2020-10-01 2021-05-01 huc10
done 12 2020-10-01 2021-05-01 huc10
done 13 2020-10-01 2021-05-01 huc10
done 14 2020-10-01 2021-05-01 huc10
done 15 2020-10-01 2021-05-01 huc10
done 16 2020-10-01 2021-05-01 huc10
done 17 2020-10-01 2021-05-01 huc10
done 18 2020-10-01 2021-05-01 huc10
done 19 2020-10-01 2021-05-01 huc10
done 20 2020-10-01 2021-05-01 huc10
done 21 2020-10-01 2021-05-01 huc10
done 22 2020-10-01 2021-05-01 huc10
done 23 2020-10-01 2021-05-01 huc10
done 24 2020-10-01 2021-05-01 huc10
done 25 2020-10-01 2021-05-01 huc10
done 26 2020-10-01 2021-05-01 huc10
done 27 2020-10-01 2021-05-01 huc10
done 28 2020-10-01 2021-05-01 huc10
done 29 2020-10-01 2021-05-01 huc10
done 0 2021-04-01 2021-11-01 huc10
done 1 2021-04-01 2021-11-01 huc10
done 2 2021-04-01 2021-11-01 huc10
done 3 2021-04-01 2021-11-01 huc10
done 4 2021-04-01 2021-11-01 huc10
done 5 2021-04-01 2021-11-01 huc10
done 6 2021-04-01 2021-11-01 huc10
done 7 2021-04-01 2021-11-01 huc10


In [20]:
#####HUC10
#124 Sites
datesStag = (('2013-03-18', '2013-11-01'),
        ('2013-10-01', '2014-05-01'),
        ('2014-04-01', '2014-11-01'),
        ('2014-10-01', '2015-05-01'),
        ('2015-04-01', '2015-11-01'),
        ('2015-10-01', '2016-05-01'),
        ('2016-04-01', '2016-11-01'),
        ('2016-10-01', '2017-05-01'),
        ('2017-04-01', '2017-11-01'),
        ('2017-10-01', '2018-05-01'),
        ('2018-04-01', '2018-11-01'),
        ('2018-10-01', '2019-05-01'),
        ('2019-04-01', '2019-11-01'),
        ('2019-10-01', '2020-05-01'),
        ('2020-04-01', '2020-11-01'),
        ('2020-10-01', '2021-05-01'),
        ('2021-04-01', '2021-11-01'),
        ('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc10_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(2000).slice(60,80)
huc = 'huc10'


#Call function
featTest(hucList)


done 0 2013-03-18 2013-11-01 huc10
done 1 2013-03-18 2013-11-01 huc10
done 2 2013-03-18 2013-11-01 huc10
done 3 2013-03-18 2013-11-01 huc10
done 4 2013-03-18 2013-11-01 huc10
done 5 2013-03-18 2013-11-01 huc10
done 6 2013-03-18 2013-11-01 huc10
done 7 2013-03-18 2013-11-01 huc10
done 8 2013-03-18 2013-11-01 huc10
done 9 2013-03-18 2013-11-01 huc10
done 10 2013-03-18 2013-11-01 huc10
done 11 2013-03-18 2013-11-01 huc10
done 12 2013-03-18 2013-11-01 huc10
done 13 2013-03-18 2013-11-01 huc10
done 14 2013-03-18 2013-11-01 huc10
done 15 2013-03-18 2013-11-01 huc10
done 16 2013-03-18 2013-11-01 huc10
done 17 2013-03-18 2013-11-01 huc10
done 18 2013-03-18 2013-11-01 huc10
done 19 2013-03-18 2013-11-01 huc10
done 0 2013-10-01 2014-05-01 huc10
done 1 2013-10-01 2014-05-01 huc10
done 2 2013-10-01 2014-05-01 huc10
done 3 2013-10-01 2014-05-01 huc10
done 4 2013-10-01 2014-05-01 huc10
done 5 2013-10-01 2014-05-01 huc10
done 6 2013-10-01 2014-05-01 huc10
done 7 2013-10-01 2014-05-01 huc10
done 8 201

done 11 2018-10-01 2019-05-01 huc10
done 12 2018-10-01 2019-05-01 huc10
done 13 2018-10-01 2019-05-01 huc10
done 14 2018-10-01 2019-05-01 huc10
done 15 2018-10-01 2019-05-01 huc10
done 16 2018-10-01 2019-05-01 huc10
done 17 2018-10-01 2019-05-01 huc10
done 18 2018-10-01 2019-05-01 huc10
done 19 2018-10-01 2019-05-01 huc10
done 0 2019-04-01 2019-11-01 huc10
done 1 2019-04-01 2019-11-01 huc10
done 2 2019-04-01 2019-11-01 huc10
done 3 2019-04-01 2019-11-01 huc10
done 4 2019-04-01 2019-11-01 huc10
done 5 2019-04-01 2019-11-01 huc10
done 6 2019-04-01 2019-11-01 huc10
done 7 2019-04-01 2019-11-01 huc10
done 8 2019-04-01 2019-11-01 huc10
done 9 2019-04-01 2019-11-01 huc10
done 10 2019-04-01 2019-11-01 huc10
done 11 2019-04-01 2019-11-01 huc10
done 12 2019-04-01 2019-11-01 huc10
done 13 2019-04-01 2019-11-01 huc10
done 14 2019-04-01 2019-11-01 huc10
done 15 2019-04-01 2019-11-01 huc10
done 16 2019-04-01 2019-11-01 huc10
done 17 2019-04-01 2019-11-01 huc10
done 18 2019-04-01 2019-11-01 huc10
do

In [22]:
#####HUC10
#124 Sites
datesStag = (('2013-03-18', '2013-11-01'),
        ('2013-10-01', '2014-05-01'),
        ('2014-04-01', '2014-11-01'),
        ('2014-10-01', '2015-05-01'),
        ('2015-04-01', '2015-11-01'),
        ('2015-10-01', '2016-05-01'),
        ('2016-04-01', '2016-11-01'),
        ('2016-10-01', '2017-05-01'),
        ('2017-04-01', '2017-11-01'),
        ('2017-10-01', '2018-05-01'),
        ('2018-04-01', '2018-11-01'),
        ('2018-10-01', '2019-05-01'),
        ('2019-04-01', '2019-11-01'),
        ('2019-10-01', '2020-05-01'),
        ('2020-04-01', '2020-11-01'),
        ('2020-10-01', '2021-05-01'),
        ('2021-04-01', '2021-11-01'),
        ('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc10_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(2000).slice(80,100)
huc = 'huc10'


#Call function
featTest(hucList)


done 0 2013-03-18 2013-11-01 huc10
done 1 2013-03-18 2013-11-01 huc10
done 2 2013-03-18 2013-11-01 huc10
done 3 2013-03-18 2013-11-01 huc10
done 4 2013-03-18 2013-11-01 huc10
done 5 2013-03-18 2013-11-01 huc10
done 6 2013-03-18 2013-11-01 huc10
done 7 2013-03-18 2013-11-01 huc10
done 8 2013-03-18 2013-11-01 huc10
done 9 2013-03-18 2013-11-01 huc10
done 10 2013-03-18 2013-11-01 huc10
done 11 2013-03-18 2013-11-01 huc10
done 12 2013-03-18 2013-11-01 huc10
done 13 2013-03-18 2013-11-01 huc10
done 14 2013-03-18 2013-11-01 huc10
done 15 2013-03-18 2013-11-01 huc10
done 16 2013-03-18 2013-11-01 huc10
done 17 2013-03-18 2013-11-01 huc10
done 18 2013-03-18 2013-11-01 huc10
done 19 2013-03-18 2013-11-01 huc10
done 0 2013-10-01 2014-05-01 huc10
done 1 2013-10-01 2014-05-01 huc10
done 2 2013-10-01 2014-05-01 huc10
done 3 2013-10-01 2014-05-01 huc10
done 4 2013-10-01 2014-05-01 huc10
done 5 2013-10-01 2014-05-01 huc10
done 6 2013-10-01 2014-05-01 huc10
done 7 2013-10-01 2014-05-01 huc10
done 8 201

done 11 2018-10-01 2019-05-01 huc10
done 12 2018-10-01 2019-05-01 huc10
done 13 2018-10-01 2019-05-01 huc10
done 14 2018-10-01 2019-05-01 huc10
done 15 2018-10-01 2019-05-01 huc10
done 16 2018-10-01 2019-05-01 huc10
done 17 2018-10-01 2019-05-01 huc10
done 18 2018-10-01 2019-05-01 huc10
done 19 2018-10-01 2019-05-01 huc10
done 0 2019-04-01 2019-11-01 huc10
done 1 2019-04-01 2019-11-01 huc10
done 2 2019-04-01 2019-11-01 huc10
done 3 2019-04-01 2019-11-01 huc10
done 4 2019-04-01 2019-11-01 huc10
done 5 2019-04-01 2019-11-01 huc10
done 6 2019-04-01 2019-11-01 huc10
done 7 2019-04-01 2019-11-01 huc10
done 8 2019-04-01 2019-11-01 huc10
done 9 2019-04-01 2019-11-01 huc10
done 10 2019-04-01 2019-11-01 huc10
done 11 2019-04-01 2019-11-01 huc10
done 12 2019-04-01 2019-11-01 huc10
done 13 2019-04-01 2019-11-01 huc10
done 14 2019-04-01 2019-11-01 huc10
done 15 2019-04-01 2019-11-01 huc10
done 16 2019-04-01 2019-11-01 huc10
done 17 2019-04-01 2019-11-01 huc10
done 18 2019-04-01 2019-11-01 huc10
do

In [23]:
#####HUC10
#124 Sites
datesStag = (('2013-03-18', '2013-11-01'),
        ('2013-10-01', '2014-05-01'),
        ('2014-04-01', '2014-11-01'),
        ('2014-10-01', '2015-05-01'),
        ('2015-04-01', '2015-11-01'),
        ('2015-10-01', '2016-05-01'),
        ('2016-04-01', '2016-11-01'),
        ('2016-10-01', '2017-05-01'),
        ('2017-04-01', '2017-11-01'),
        ('2017-10-01', '2018-05-01'),
        ('2018-04-01', '2018-11-01'),
        ('2018-10-01', '2019-05-01'),
        ('2019-04-01', '2019-11-01'),
        ('2019-10-01', '2020-05-01'),
        ('2020-04-01', '2020-11-01'),
        ('2020-10-01', '2021-05-01'),
        ('2021-04-01', '2021-11-01'),
        ('2021-10-01', '2022-05-01'),
        ('2022-04-01', '2022-11-01'),
        ('2022-10-01', '2023-03-01'))


#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites_filtered/aquasat_huc10_5sitelatlong_2')
hucList = ee.FeatureCollection(hucSites).toList(2000).slice(100,124)
huc = 'huc10'


#Call function
featTest(hucList)


done 0 2013-03-18 2013-11-01 huc10
done 1 2013-03-18 2013-11-01 huc10
done 2 2013-03-18 2013-11-01 huc10
done 3 2013-03-18 2013-11-01 huc10
done 4 2013-03-18 2013-11-01 huc10
done 5 2013-03-18 2013-11-01 huc10
done 6 2013-03-18 2013-11-01 huc10
done 7 2013-03-18 2013-11-01 huc10
done 8 2013-03-18 2013-11-01 huc10
done 9 2013-03-18 2013-11-01 huc10
done 10 2013-03-18 2013-11-01 huc10
done 11 2013-03-18 2013-11-01 huc10
done 12 2013-03-18 2013-11-01 huc10
done 13 2013-03-18 2013-11-01 huc10
done 14 2013-03-18 2013-11-01 huc10
done 15 2013-03-18 2013-11-01 huc10
done 16 2013-03-18 2013-11-01 huc10
done 17 2013-03-18 2013-11-01 huc10
done 18 2013-03-18 2013-11-01 huc10
done 19 2013-03-18 2013-11-01 huc10
done 20 2013-03-18 2013-11-01 huc10
done 21 2013-03-18 2013-11-01 huc10
done 22 2013-03-18 2013-11-01 huc10
done 23 2013-03-18 2013-11-01 huc10
done 0 2013-10-01 2014-05-01 huc10
done 1 2013-10-01 2014-05-01 huc10
done 2 2013-10-01 2014-05-01 huc10
done 3 2013-10-01 2014-05-01 huc10
done 4

done 15 2017-10-01 2018-05-01 huc10
done 16 2017-10-01 2018-05-01 huc10
done 17 2017-10-01 2018-05-01 huc10
done 18 2017-10-01 2018-05-01 huc10
done 19 2017-10-01 2018-05-01 huc10
done 20 2017-10-01 2018-05-01 huc10
done 21 2017-10-01 2018-05-01 huc10
done 22 2017-10-01 2018-05-01 huc10
done 23 2017-10-01 2018-05-01 huc10
done 0 2018-04-01 2018-11-01 huc10
done 1 2018-04-01 2018-11-01 huc10
done 2 2018-04-01 2018-11-01 huc10
done 3 2018-04-01 2018-11-01 huc10
done 4 2018-04-01 2018-11-01 huc10
done 5 2018-04-01 2018-11-01 huc10
done 6 2018-04-01 2018-11-01 huc10
done 7 2018-04-01 2018-11-01 huc10
done 8 2018-04-01 2018-11-01 huc10
done 9 2018-04-01 2018-11-01 huc10
done 10 2018-04-01 2018-11-01 huc10
done 11 2018-04-01 2018-11-01 huc10
done 12 2018-04-01 2018-11-01 huc10
done 13 2018-04-01 2018-11-01 huc10
done 14 2018-04-01 2018-11-01 huc10
done 15 2018-04-01 2018-11-01 huc10
done 16 2018-04-01 2018-11-01 huc10
done 17 2018-04-01 2018-11-01 huc10
done 18 2018-04-01 2018-11-01 huc10
do

done 6 2022-10-01 2023-03-01 huc10
done 7 2022-10-01 2023-03-01 huc10
done 8 2022-10-01 2023-03-01 huc10
done 9 2022-10-01 2023-03-01 huc10
done 10 2022-10-01 2023-03-01 huc10
done 11 2022-10-01 2023-03-01 huc10
done 12 2022-10-01 2023-03-01 huc10
done 13 2022-10-01 2023-03-01 huc10
done 14 2022-10-01 2023-03-01 huc10
done 15 2022-10-01 2023-03-01 huc10
done 16 2022-10-01 2023-03-01 huc10
done 17 2022-10-01 2023-03-01 huc10
done 18 2022-10-01 2023-03-01 huc10
done 19 2022-10-01 2023-03-01 huc10
done 20 2022-10-01 2023-03-01 huc10
done 21 2022-10-01 2023-03-01 huc10
done 22 2022-10-01 2023-03-01 huc10
done 23 2022-10-01 2023-03-01 huc10


In [13]:
####HUC 18
#48 sites

ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

#Asset to list and slice into smaller chunks for prediction
hucSites = ee.FeatureCollection('projects/fusion-353005/assets/fusionSites/aquasat_huc18_5sitelatlong')
hucList = ee.FeatureCollection(hucSites).toList(2000)

#Call Function
huc = '_huc18'
            
featTest(hucList.slice(0,10))

19


KeyboardInterrupt: 

In [ ]:
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

featTest(hucList.slice(10,20))

ee.Reset()
time.sleep(60 * 5)


In [ ]:
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

featTest(hucList.slice(20,30))

ee.Reset()
time.sleep(60 * 5)



In [ ]:
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

featTest(hucList.slice(30,40))

ee.Reset()
time.sleep(60 * 5)



In [ ]:
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

featTest(hucList.slice(40,50))

ee.Reset()
time.sleep(60 * 5)


In [ ]:
ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

featTest(hucList.slice(50,68))

ee.Reset()